In [16]:
###################
# Hirogen
###################
# Hunter of tIdal disRuptiOn events in Galaxies with coronal linE emissioN
###################

import sys
import time
import warnings
import math
import os
import Hirogen_Functions
import pkg_resources

import numpy as np
import scipy.constants as constants
import matplotlib.pyplot as plt

from astropy import units as u
from datetime import datetime, date
from glob import glob
from astropy.io import fits
from astropy.table import Table, vstack, hstack, join
from tqdm.notebook import tqdm
from scipy.ndimage import gaussian_filter1d
from IPython.display import clear_output

from astropy.table.pprint import conf
conf.max_lines = -1
conf.max_width = -1

np.seterr(invalid='raise', divide='raise')

#######################
# NERSC Flag
#######################

# Flags the script as running on NERSC - i.e. on DESI spectra
# This activates changes in the script to handle the DESI spectra as well as disabling any multithreading for the
# time being. Also changes the import paths to the proper paths

NERSC_Flag = False # True - Running on NERSC, FALSE - Running locally
Local_DESI = False
# Both flags should be False to run on local SDSS spectra

if not NERSC_Flag:

    User = 'Joe'
    User_Config = Hirogen_Functions.user_config(user=User)

    #TableID = "SDSS_Test_QSO_Sample"
    #TableID = "SDSS_Test_Sample"
    TableID = "SDSS_Fake_Spectra"
    #TableID = "SDSS_Confirmed_Objects"
    #TableID = 'DESI_Local'

    # Search for 'QUERY' to find the actual database access location where parameters can be adjusted

    Database = User_Config[0]
    Database_User = User_Config[1]
    Database_Password = User_Config[2]
    Main_Spectra_Path = User_Config[3]

    from mpi4py import MPI

    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    comm.Barrier()
    
else:
    
    pkg_resources.require("desispec>=0.47.0")

    import matplotlib
    import desispec

    import Hirogen_DESI_Functions # Unique code required for running on DESI data via NERSC

    from desitarget.cmx.cmx_targetmask import cmx_mask
    from desitarget.targetmask import desi_mask # Main DESI Mask
    from desitarget.sv1.sv1_targetmask import bgs_mask # < This is the mask for BGS Targets in SV1

    from desispec.io import read_spectra, write_spectra, specprod_root
    from desispec.coaddition import coadd_cameras

    from collections import defaultdict

    from astropy.nddata import InverseVariance

    sv1_bgs_bits = '|'.join([_ for _ in bgs_mask.names() if 'BGS' in _])
    sv3_bgs_bits = '|'.join([_ for _ in bgs_mask.names() if 'BGS' in _])
    main_bgs_bits = '|'.join([_ for _ in desi_mask.names() if 'BGS' in _])

startT = time.time()

########################
# General Configuration
########################

debug = False
header_print = False

Fakes = True # True if fake spectra are being used

c = constants.value('speed of light in vacuum') / 1000  # Speed of light in kms^-1

config_parameters = Hirogen_Functions.main_config()  # Draws from centralised parameter declarations

# These four parameters control what regions of the spectra are 'cut out' for analysis around each line
Lower_Wave = config_parameters[0]
Upper_Wave = config_parameters[1]

Lower_Shift = config_parameters[2]
Upper_Shift = config_parameters[3]

# Any object with a candidate score equal to or exceeding this value will be treated as an ECLE candidate
Candidate_Threshold = config_parameters[4]

# Maximum possible candidate score
Candidate_Score_Max = config_parameters[5]

# To qualify as a line detection for candidate selection the overall feature pEQW must exceed this value:
LineDetection_pEQW_Threshold = config_parameters[6]
LineDetection_Max_Threshold = config_parameters[7]

LineDetection_Max_Above_Av_Continua_Threshold = config_parameters[8]

# To qualify as a line detection for candidate selection the maximum flux point of the feature must occur
# within this +_ kms^-1 of the zero velocity point
LineDetection_Peak_Tolerance = config_parameters[9]

# The region inside which the peak max is considered - a peak outside this region will not be considered to be the
# true line peak. Helps filter out where lines are closely spaced with potentially unrelated features (kms^-1)
Line_Peak_Location_Region = config_parameters[10]

# If the minimum of the line peak region falls below this value the line cannot score - designed to weed out very
# noisy lines
Line_Peak_Region_Minima_Threshold = config_parameters[11]

# Thresholds for identifying strong lines. First is overall line strength, second is for detecting sharp peaks
Strong_EQW_Threshold = config_parameters[12]
Strong_Peak_Max_Threshold = config_parameters[13]

# Applies a boxcar smoothing function of width to the spectrum on read in
# There is an override flag in the database to force a spectrum to not be smoothed if needed on a per-spectrum basis
Default_Smoothing = True # Only applies to SDSS spectra

########################
# Line 'list' Dictionaries
########################

# These lists control which lines are explored

Spectral_Lines_For_Analysis_Air = Hirogen_Functions.lines_for_analysis() # All lines here are analysed
Spectral_Lines_For_Analysis_Air_Names = list(Spectral_Lines_For_Analysis_Air.keys()) # Display ready line labels

Spectral_Lines_For_Scoring_Air = Hirogen_Functions.lines_for_scoring() # All lines here factor into the ECLE scoring

########################
# Output Directories Setup
########################

Timestamp = str(datetime.today().date())

# Configuration and generation of output directory savepaths
MainSavePath = os.path.join(f"Hirogen_Outputs/{Timestamp}")
Hirogen_Functions.directory_creation(MainSavePath)

MainPicklePath = 'PerBlockPickleFiles'
Pickle_Dump = False # The flag for activating the dump to pickle files


if NERSC_Flag:
    
    print("Main output directory has been created - sub directories will be generated later")
    Hirogen_Functions.directory_creation(MainPicklePath)
    
    print("NERSC Flag Set: Configuring Environment Variables")

    # Setting the environment paths to the survey verification data
    os.environ["DESI_SPECTRO_REDUX"]="/global/cfs/cdirs/desi/spectro/redux/daily"
    os.environ["SPECPROD"]="tiles"

    reduxdir = specprod_root()

    # Checking that the environment variables have been set correctly
    Hirogen_Functions.DESI_check_env()

    #spectra_basedir = os.path.join(os.getenv("DESI_SPECTRO_REDUX"), os.getenv("SPECPROD"), "cumulative")
    #spectra_subdir = sorted(os.listdir(spectra_basedir))

    #print(f"Selected Data Directory:{spectra_basedir}")
    #print(f"Available subdirectories:{spectra_subdir}\n")
    
    #DESI Specific Variables
    date = "20201222" #20200226, 20200227 - Appears to be no data, 20200228 - Error with z file, 20200301, 20200302 - No data
    #20200219 - Has data
    #20200220, 20210428, 20210429, 20210430 - Observations but no BGS targets
    
    # DESI Survey Settting - Only used in the NERSC Flag is active
    # This alters the data read in settings to take into account the formats of the various surveys
    # Currently, SV1, SV3, Main and Everest
    
    Survey = 'Everest'
    
    def get_petal_id(filename):
        return int(filename.split('-')[1])

    def match_files(cafiles, zbfiles):  
        matched = [] 
        for _l1 in cafiles: 
            p1 = get_petal_id(_l1)  
            for _l2 in zbfiles: 
                p2 = get_petal_id(_l2) 
                if p1 == p2: 
                    matched.append([_l1, _l2]) 
                    break
        return matched 
    
    SummaryPlots_SavePath = os.path.join(MainSavePath, f"Summary_Plots")
    Hirogen_Functions.directory_creation(SummaryPlots_SavePath)
    Survey = Survey.upper()

print('Initial configuration complete')

Initial configuration complete


If on NERSC - Runs a check to display the current versions of desispec, numpy and matplotlib to confirm the kernel is properly updated

In [17]:
if NERSC_Flag:
    print(f'DESISPEC Version : {desispec.__version__}')
    print(f'Numpy Version : {np.__version__}')
    print(f'Matplotlib Version : {matplotlib.__version__}')
else:
    print('Not on NERSC: Package version check not active')

Not on NERSC: Package version check not active


In [18]:
########################
# Data entry
########################

# If not running on NERSC - hooks into a mySQL database
# If running on NERSC will draw information directly from the appropriate DESI files directly rather than the database

########################
# Smoothing and Denoise configuration
########################

# Recommended Configuration:

# SDSS: Smoothing = True, FFT_Denoise = False, Gaussian_Smooth = False, Median_Filter = False 
# DESI: 
# Either: Smoothing = False, FFT_Denoise = True, Gaussian_Smooth = True, Median_Filter = False < More aggressive
# Or: Smoothing = False, FFT_Denoise = True, Gaussian_Smooth = False, Median_Filter = True

Spectres_Rebin = False
Rebin_Res = 5

Smoothing = True # Boxcar Smoothing
Boxcar_Size = 5

FFT_Denoise = False  # Applies a FFT to reduce the erratic noise on each pixel
FFT_Threshold_Str = '1e5' # Is needed to get round the database storage issue
FFT_Threshold = 1e5

Gaussian_Smooth = False # Controls if a gaussian smoothing factor is used
Gaussian_Sigma = 3

Median_Filter = False # Defaults for DESI spectra: No boxcar smoothing but has a median filter applied
Median_Filter_Kernel = 3

Settings_Config = f'{int(Spectres_Rebin)}_{int(Rebin_Res)}_{int(Smoothing)}_{int(FFT_Denoise)}_{FFT_Threshold_Str}_{int(Gaussian_Smooth)}_{Gaussian_Sigma}_{int(Median_Filter)}_{Median_Filter_Kernel}'
# Rolls all the smoothing / denoise configuration settings into one string for storage and comparison

Exception_Values = [-1000, -999, -995, -9999, 0]
# Values that indicate where lines are not accessible etc. - Currently only used to exclude values from the line ratio calculation

if NERSC_Flag:
    #BLOCK_ID = 'Everest_Main_Bright_1'
    #Tile_Numbers = [20000, 20001, 20004, 20005, 20007]
    
    BLOCK_ID = 'Everest_Main_Dark_1_Test'
    Tile_Numbers = [1000,1001,1007,1010,1030]

    
print('Processing Config Complete')

Processing Config Complete


This cell runs the analysis if NOT on NERSC

In [19]:
if NERSC_Flag:
    print('On NERSC: Moving to NERSC only analysis cell')

else:

    if not Local_DESI:

        ########################
        # Object Info - From mySQL Database
        ########################

        Data = Hirogen_Functions.database_connection(user=Database_User, password=Database_Password, database=Database)

        print("\n")
        cursor = Data.cursor()

        # The 'errors' here are because PyCharm can't seem to cope with the Table name being variable rather than a normal sting - it will run properly though.
        # To test is the schema is right swap out the variable for the table name temporarily

        # Can be set to only run the analysis for specific IDs given in the following format
        IDs = (1021306429161629696)
        #IDs = (8587366798911098880, 8598733000363233280)

        # Manually_Inspected_Flag = -10 are spectra with external issues that prevent their use

        # QUERY
        cursor.execute(
            f"SELECT DR16_Spectroscopic_ID, spec_Plate, spec_MJD, spec_FiberID, z_SDSS_spec, generalised_extinction, "
            f"survey, run2d, Standard_Inclusion,Path_Override_Flag, Path_Override, Follow_Up_ID, Smoothing_Override,"
            f"z_corrected_flag, extinction_corrected_flag "
            f"FROM `{Database}`.`{TableID}`"
            #f"WHERE Manually_Inspected_Flag != -10 AND lin_con_LineFlux_Halpha is NULL"
            #f"WHERE lin_con_pEQW_Halpha is NULL"
            #f"WHERE z_SDSS_spec >= 0.2"
            #f"WHERE Follow_Up_ID = 0"
            #f"WHERE Follow_Up_ID in (1,2,3)"
            #f"WHERE Nickname = 'Leafeon' AND Follow_Up_ID in (2,3)"
            #f"WHERE Standard_Inclusion = 1"
            #f"WHERE DR16_Spectroscopic_ID IN {IDs} and Standard_Inclusion = 1"
            #f"WHERE DR16_Spectroscopic_ID = {IDs}"
        )

        Candidate_Data = cursor.fetchall()
        # Candidate_Data = cursor.fetchmany(200)

        #print(Candidate_Data)

        if len(Candidate_Data) >= 1:

            Object_Name_List = [item[0] for item in Candidate_Data]
            Plate_List = [item[1] for item in Candidate_Data]
            MJD_List = [item[2] for item in Candidate_Data]
            FiberID_List = [item[3] for item in Candidate_Data]
            Redshift_List = [item[4] for item in Candidate_Data]
            Extinction_List = [item[5] for item in Candidate_Data]
            Survey_List = [item[6] for item in Candidate_Data]
            run2d_List = [item[7] for item in Candidate_Data]
            Standard_Inclusion_List = [item[8] for item in Candidate_Data]
            Path_Override_Flag_List = [item[9] for item in Candidate_Data]
            Path_Override_List = [item[10] for item in Candidate_Data]
            Follow_Up_ID_List = [item[11] for item in Candidate_Data]
            Smoothing_Override_List = [item[12] for item in Candidate_Data]
            z_Correction_Override_List = [item[13] for item in Candidate_Data]
            Extinction_Correction_Override_List = [item[14] for item in Candidate_Data]

        else:
            print("Candidate_Data Length error: Check and try again.")

            sys.exit()

        if Fakes:
            FilePaths = Hirogen_Functions.sdss_peaks_spectra_file_path_generator(
                Main_Spectra_Path, Plate_List, MJD_List, FiberID_List, Survey_List, run2d_List,
            override_path_flag_list=Path_Override_Flag_List, override_path_list=Path_Override_List, Mode = 'Fakes'
            )
        else:
            FilePaths = Hirogen_Functions.sdss_spectra_file_path_generator(
                Main_Spectra_Path, Plate_List, MJD_List, FiberID_List, Survey_List, run2d_List,
                override_path_flag_list=Path_Override_Flag_List, override_path_list=Path_Override_List
            )


        print(FilePaths[0])

        Object_Number = 0
        Total_Object_Count = int(len(Object_Name_List))
        perrank = int(Total_Object_Count / size)

    else: # Local DESI Files - Pulls data from a database table

        ########################
        # Object Info - From mySQL Database
        ########################


        Data = Hirogen_Functions.database_connection(user=Database_User, password=Database_Password, database=Database)

        print("\n")
        cursor = Data.cursor()

        # The 'errors' here are because PyCharm can't seem to cope with the Table name being variable rather than a
        # normal string - it will run properly though. To test is the schema is right swap out the variable for the
        # table name temporarily

        # Can be set to only run the analysis for specific IDs given in the following format
        #IDs = (13101022585281597440, 300697765476853760)
        #IDs = (8587366798911098880, 8598733000363233280)

        # Manually_Inspected_Flag = -10 are spectra with external issues that prevent their use

        # QUERY
        cursor.execute(
            f"SELECT DESI_TARGET_ID, z_best, EBV, Standard_Inclusion, Path_Override_Flag, Path_Override, Follow_Up_ID, "
            f"Smoothing_Override  "
            f"FROM `{Database}`.`{TableID}`"
            #f"WHERE Manually_Inspected_Flag != -10 AND lin_con_LineFlux_Halpha is NULL"
            #f"WHERE lin_con_pEQW_Halpha is NULL"
            #f"WHERE z_best >= 0.2"
            f"WHERE Follow_Up_ID = 0"
            # f"WHERE Follow_Up_ID in (1,2,3)"
            #f"WHERE Standard_Inclusion = 1"
            # f"WHERE DR16_Spectroscopic_ID IN {IDs} and Standard_Inclusion = 1"
        )

        Candidate_Data = cursor.fetchall()
        # Candidate_Data = cursor.fetchmany(200)

        #print(Candidate_Data)

        if len(Candidate_Data) >= 1:

            Object_Name_List = [item[0] for item in Candidate_Data]
            Redshift_List = [item[1] for item in Candidate_Data]
            Extinction_List = [item[2] for item in Candidate_Data]
            Standard_Inclusion_List = [item[3] for item in Candidate_Data]
            Path_Override_Flag_List = [item[4] for item in Candidate_Data]
            Path_Override_List = [item[5] for item in Candidate_Data]
            Follow_Up_ID_List = [item[6] for item in Candidate_Data]
            Smoothing_Override_List = [item[7] for item in Candidate_Data]

        else:
            print("Candidate_Data Length error: Check and try again.")

            sys.exit()

        Total_Object_Count = len(Object_Name_List)

    #################################
    # Counting Variables
    #################################

    Successful_Objects_Count = 0
    Failed_Objects_Count = 0
    Candidate_Count = 0
    ObjectCount = 0
    Strong_FeVII_Object_Count = 0
    Strong_FeX_Object_Count = 0
    Strong_FeXI_Object_Count = 0
    Strong_FeXIV_Object_Count = 0

    Failed_NII_over_Halpha_Line_Ratio = []
    Failed_OIII_over_Hbeta_Line_Ratio = []
    Failed_SII_over_Halpha_Line_Ratio = []
    Failed_OI_over_Halpha_Line_Ratio = []
    Failed_OIII5007_over_OIII4959_Ratio = []
    Failed_HeII_over_Halpha_Ratio = []
    Failed_Lick = []

    # Disabled multithreading activation code
    """
    perrank = int(Total_Objects_Count / size)

    for Object_Number in tqdm(range(rank * perrank, (rank + 1) * perrank),
                              desc="Number of spectra processed"):
    """

    if Local_DESI:
        z_Correction_Override_List = [np.float64(0)] * Total_Object_Count
        Extinction_Correction_Override_List = [np.float64(0)] * Total_Object_Count

    print(Total_Object_Count)
    Spectrum_Number = 0

    for Spectrum_Number in tqdm(range(Total_Object_Count)):

        Candidate_Score = 0
        Strong_FeVII_Flag = 0
        Strong_FeX_Flag = 0
        Strong_FeXI_Flag = 0
        Strong_FeXIV_Flag = 0

        if not Local_DESI:

            ObjectName = Object_Name_List[Spectrum_Number]
            print(ObjectName)

            if Smoothing_Override_List[Spectrum_Number] == 1:
                Smoothing = False
            else:
                Smoothing = Default_Smoothing

            if debug:
                print("\n\n")
                print(f"{ObjectName}")

            if Standard_Inclusion_List[Object_Number] == 0:

                FilePaths[Spectrum_Number] = Path_Override_List[Spectrum_Number]

                # I'm going to assume that non-standard files are .txt or .ascii with three columns
                # wave, flux, flux err, unless specifically told otherwise via the 'survey' column in the database
                # wavelength should be given in angstroms

                if Survey_List[Spectrum_Number] in ('MMT_Fits', 'Fits', 'FITS'):

                        Spectral_Data = Hirogen_Functions.fits_spectrum_reader(
                        filepath=FilePaths[Spectrum_Number],
                        z=Redshift_List[Spectrum_Number],
                        extinction=Extinction_List[Spectrum_Number],
                        smoothing=Smoothing,
                        z_correction_flag=z_Correction_Override_List[Spectrum_Number],
                        extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number]
                    )

                else:

                    Spectral_Data = Hirogen_Functions.ascii_spectrum_reader(
                        filepath=FilePaths[Spectrum_Number],
                        z=Redshift_List[Spectrum_Number],
                        extinction=Extinction_List[Spectrum_Number],
                        smoothing=Smoothing,
                        z_correction_flag=z_Correction_Override_List[Spectrum_Number],
                        extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number]
                    )
            else:
                Spectral_Data = Hirogen_Functions.sdss_spectrum_reader(
                    filepath=FilePaths[Spectrum_Number],
                    z=Redshift_List[Spectrum_Number],
                    extinction=Extinction_List[Spectrum_Number],
                    smoothing=Smoothing,
                    z_correction_flag=z_Correction_Override_List[Spectrum_Number],
                    extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number]
                )


            wave = Spectral_Data[0]
            flux = Spectral_Data[1]
            error = Spectral_Data[2]

            if Survey_List[Spectrum_Number] == 'DESI': #DESI spectra have the multiplication factor built in
                flux = flux / 1e17
                error = error / 1e17

            All_Flags_Percentage = Spectral_Data[6]
            Counted_Flags_Percentage = Spectral_Data[7]

            ###########
            # Rebinning
            ###########

            if Spectres_Rebin:
                spectres_data = Hirogen_Functions.spectres_rebin(flux=flux.value, wave=wave.value, desired_res=Rebin_Res)
                flux = spectres_data[0] * u.Unit('erg cm-2 s-1 AA-1')
                wave = spectres_data[1] * u.Unit('AA')

            ###########
            # Other, non-boxcar Smoothing
            ###########

            if FFT_Denoise:
                filtered_flux = Hirogen_Functions.fft_filter_signal(flux.value, threshold=FFT_Threshold)
                flux = filtered_flux * u.Unit('erg cm-2 s-1 AA-1')

            if Gaussian_Smooth:
                gaussian_smoothed_flux = Hirogen_Functions.gaussian_smoothing(flux.value, sigma_value = Gaussian_Sigma)
                flux = gaussian_smoothed_flux * u.Unit('erg cm-2 s-1 AA-1')

        else: # Local DESI Files

            Smoothing = False

            ObjectName = Object_Name_List[Spectrum_Number]

            All_Flags_Percentage = -999
            Counted_Flags_Percentage = -999
            # DESI flagging is currently not yet implemented

            Spectral_Data = Hirogen_Functions.ascii_spectrum_reader(
            filepath = Path_Override_List[Spectrum_Number],
            z=Redshift_List[Spectrum_Number],
            extinction=Extinction_List[Spectrum_Number],
            smoothing=Smoothing,
            z_correction_flag=z_Correction_Override_List[Spectrum_Number],
            extinction_correction_flag=Extinction_Correction_Override_List[Spectrum_Number])

            wave = Spectral_Data[0]
            flux = Spectral_Data[1] / 1e17
            error = Spectral_Data[2] / 1e17

            # Divsisions remove the extra SDSS equivilence factor

        ###########
        # Rebinning
        ###########

        if Spectres_Rebin:
            spectres_data = Hirogen_Functions.spectres_rebin(flux=flux.value, wave=wave.value, desired_res=Rebin_Res)
            flux = spectres_data[0] * u.Unit('erg cm-2 s-1 AA-1')
            wave = spectres_data[1] * u.Unit('AA')

        ###########
        # Smoothing
        ###########

        if Smoothing:  # If active runs a boxcar smooth, defaults to a width of 5 but can be set via the function call
            smoothed_flux = Hirogen_Functions.spectral_flux_smooth(flux)
            flux = smoothed_flux

        ###########
        # Other, non-boxcar Smoothing
        ###########

        if FFT_Denoise:
            filtered_flux = Hirogen_Functions.fft_filter_signal(flux.value, threshold=FFT_Threshold)
            flux = filtered_flux * u.Unit('erg cm-2 s-1 AA-1')

        if Median_Filter: # Runs a median filter, defaults to a kernel of size 3 but can be set via the function call
            med_filtered_flux = Hirogen_Functions.spectral_median_filter(flux, med_filter_kernel= Median_Filter_Kernel)
            flux = med_filtered_flux * u.Unit('erg cm-2 s-1 AA-1')

        if Gaussian_Smooth:
            gaussian_smoothed_flux = Hirogen_Functions.gaussian_smoothing(flux.value, sigma_value = Gaussian_Sigma)
            flux = gaussian_smoothed_flux * u.Unit('erg cm-2 s-1 AA-1')

        ######
        # General check and removal of -ve flux points
        ######

        """
        for ii, item in enumerate(flux):
            if flux[ii].value < 0:
                flux[ii] = np.nan * u.Unit('erg cm-2 s-1 AA-1')
        """

        #######
        # Spectral readin + processing for one spectrum completed
        #######

        ###########
        # Line pEQW measurements with linear continuum subtraction
        ###########

        for ii, item in enumerate(Spectral_Lines_For_Analysis_Air):

            Line_Location = Spectral_Lines_For_Analysis_Air[item][0]

            # Velocity shift from line
            Shift = (((np.array(wave) * c) / Line_Location) - c)

            # Cut down the region for analysis
            # This may need to be tweaked currently 75 angstroms either side of the line
            # Will default to nans if spectrum does not extend to fully include these regions

            if wave[-1].value < Line_Location - 75 or wave[1].value > Line_Location + 75:

                if debug:
                    print(f"Analysis region \n{Spectral_Lines_For_Analysis_Air_Names[ii]} "
                          f"is not fully covered in the rest frame.")

                Spectral_Lines_For_Analysis_Air[item][2] = np.nan
                Spectral_Lines_For_Analysis_Air[item][3] = np.nan
                Spectral_Lines_For_Analysis_Air[item][4] = np.nan
                Spectral_Lines_For_Analysis_Air[item][5] = np.nan
                Spectral_Lines_For_Analysis_Air[item][6] = np.nan
                Spectral_Lines_For_Analysis_Air[item][7] = np.nan
                Spectral_Lines_For_Analysis_Air[item][8] = -999
                Spectral_Lines_For_Analysis_Air[item][9] = np.nan
                Spectral_Lines_For_Analysis_Air[item][10] = np.nan
                Spectral_Lines_For_Analysis_Air[item][11] = np.nan
                Spectral_Lines_For_Analysis_Air[item][16] = -999

            else:
                Shift_Region, Wave_Region, Flux_Region, Error_Region = Hirogen_Functions.region_cutter(
                    shift=Shift, wave=wave,
                    flux=flux,
                    low_cut=Line_Location - Lower_Wave,
                    high_cut=Line_Location + Upper_Wave,
                    mode='Wavelength'
                )

                # Generate and remove a continuum fit from the flux data - currently linear other functions seem worse
                # The scaled flux output - Continuum[1] - is scaled to a continuum value of 1
                # The returned continuum values - Continuum[0] - is relative to the original unscaled spectrum
                Continuum = Hirogen_Functions.continua_maker(
                    spec_region=Wave_Region,
                    flux=Flux_Region,
                    # shift=Shift_Region,
                    line_name=list(Spectral_Lines_For_Analysis_Air)[ii],
                    line_loc=Line_Location,
                    object_name=ObjectName
                )

                # Measure the resulting pEQW of the feature

                pEQW_LinCon_Spec, Line_Flux = Hirogen_Functions.eqw_measurement(
                    flux = Flux_Region,
                    true_continuum = Continuum[0],
                    scaled_flux=Continuum[1],
                    xaxis=Wave_Region,
                    line_loc=Line_Location
                )

                # Store the results in the main dictionary

                Spectral_Lines_For_Analysis_Air[item][2] = Shift
                Spectral_Lines_For_Analysis_Air[item][3] = Shift_Region
                Spectral_Lines_For_Analysis_Air[item][4] = Wave_Region
                Spectral_Lines_For_Analysis_Air[item][5] = Flux_Region
                Spectral_Lines_For_Analysis_Air[item][6] = Continuum[0]
                Spectral_Lines_For_Analysis_Air[item][7] = Continuum[1]

                if np.isnan(pEQW_LinCon_Spec):
                    Spectral_Lines_For_Analysis_Air[item][8] = -9999
                else:
                    Spectral_Lines_For_Analysis_Air[item][8] = pEQW_LinCon_Spec

                if np.isnan(Line_Flux):
                    Spectral_Lines_For_Analysis_Air[item][16] = -9999
                else:
                    Spectral_Lines_For_Analysis_Air[item][16] = Line_Flux

                ##############################
                # Candidate Conditions Checks
                ##############################

                if Spectral_Lines_For_Analysis_Air_Names[ii] in Spectral_Lines_For_Scoring_Air:

                    LineScore = Hirogen_Functions.lin_con_candidate_line_identifier(
                        line_name=Spectral_Lines_For_Analysis_Air_Names[ii],
                        line_peqw=Spectral_Lines_For_Analysis_Air[item][8],
                        peqw_threshold=LineDetection_pEQW_Threshold,
                        continuum_removed_flux=Continuum[1],
                        shift_points=Shift_Region,
                        peak_threshold=LineDetection_Max_Threshold,
                        above_continuum_threshold=LineDetection_Max_Above_Av_Continua_Threshold,
                        peak_max_region=LineDetection_Peak_Tolerance,
                        peak_region=Line_Peak_Location_Region,
                        feature_region=3000,
                        peak_region_minima_threshold=Line_Peak_Region_Minima_Threshold
                    )
                    Candidate_Score = Candidate_Score + LineScore[0]
                    Spectral_Lines_For_Analysis_Air[item][9] = LineScore[4]
                    Spectral_Lines_For_Analysis_Air[item][10] = LineScore[2]
                    Spectral_Lines_For_Analysis_Air[item][11] = LineScore[7]
                else:
                    Spectral_Lines_For_Analysis_Air[item][9] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][10] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][11] = np.nan

        if Spectral_Lines_For_Analysis_Air['[FeVII]6088'][8] < Strong_EQW_Threshold and \
                Spectral_Lines_For_Analysis_Air['[FeVII]6088'][9] > Strong_Peak_Max_Threshold:
            Strong_FeVII_Flag = 1
            Strong_FeVII_Object_Count += 1

        if Spectral_Lines_For_Analysis_Air['[FeX]6376'][8] < Strong_EQW_Threshold and \
                Spectral_Lines_For_Analysis_Air['[FeX]6376'][9] > Strong_Peak_Max_Threshold:
            Strong_FeX_Flag = 1
            Strong_FeX_Object_Count += 1

        if Spectral_Lines_For_Analysis_Air['[FeXI]7894'][8] < Strong_EQW_Threshold and \
                Spectral_Lines_For_Analysis_Air['[FeXI]7894'][9] > Strong_Peak_Max_Threshold:
            Strong_FeXI_Flag = 1
            Strong_FeXI_Object_Count += 1

        if Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][8] < Strong_EQW_Threshold and \
                Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][9] > Strong_Peak_Max_Threshold:
            Strong_FeXIV_Flag = 1
            Strong_FeXIV_Object_Count += 1

        if Spectral_Lines_For_Analysis_Air['[NII]6584'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Halpha'][16] in Exception_Values:
            NII_over_Halpha_Ratio_LinCon = -1000
        else:
            try:
                NII_over_Halpha_Ratio_LinCon = np.log10(
                    Spectral_Lines_For_Analysis_Air['[NII]6584'][16] / Spectral_Lines_For_Analysis_Air['Halpha'][16])
            except FloatingPointError:
                if debug:
                    print("\nlog10 error in the determination of NII_over_Halpha_Ratio_LinCon:\n"
                          "Likely cause is one or both lines measured in absorption."
                          "\nSetting value to -1000 and continuing.")
                NII_over_Halpha_Ratio_LinCon = -1000
                Failed_NII_over_Halpha_Line_Ratio.append(ObjectName)
            except RuntimeWarning:
                if debug:
                    print("\nlog10 error in the determination of NII_over_Halpha_Ratio_LinCon:\n"
                          "Looks like a zero got in there somewhere"
                          "\nSetting value to -1000 and continuing.")
                NII_over_Halpha_Ratio_LinCon = -1000
                Failed_NII_over_Halpha_Line_Ratio.append(ObjectName)

        if Spectral_Lines_For_Analysis_Air['[OIII]5007'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Hbeta'][16] in Exception_Values:
            OIII_over_Hbeta_Ratio_LinCon = -1000
        else:
            try:
                OIII_over_Hbeta_Ratio_LinCon = np.log10(
                    Spectral_Lines_For_Analysis_Air['[OIII]5007'][16] / Spectral_Lines_For_Analysis_Air['Hbeta'][16])
            except FloatingPointError:
                if debug:
                    print("\nlog10 error in the determination of OIII_over_Hbeta_Ratio_LinCon:\n"
                          "Likely cause is one or both lines measured in absorption."
                          "\nSetting value to -1000 and continuing.")
                OIII_over_Hbeta_Ratio_LinCon = -1000
                Failed_OIII_over_Hbeta_Line_Ratio.append(ObjectName)
            except RuntimeWarning:
                if debug:
                    print("\nlog10 error in the determination of OIII_over_Hbeta_Ratio_LinCon:\n"
                          "Looks like a zero got in there somewhere"
                          "\nSetting value to -1000 and continuing.")
                OIII_over_Hbeta_Ratio_LinCon = -1000
                Failed_OIII_over_Hbeta_Line_Ratio.append(ObjectName)

        if Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Halpha'][16] in Exception_Values:
            SII_over_Halpha_Ratio_LinCon = -1000
        else:
            try:
                SII_over_Halpha_Ratio_LinCon = np.log10(
                    Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][16] / Spectral_Lines_For_Analysis_Air['Halpha'][16])
            except FloatingPointError:
                if debug:
                    print("\nlog10 error in the determination of SII_over_Halpha_Ratio_LinCon:\n"
                          "Likely cause is one or both lines measured in absorption."
                          "\nSetting value to -1000 and continuing.")
                SII_over_Halpha_Ratio_LinCon = -1000
                Failed_SII_over_Halpha_Line_Ratio.append(ObjectName)
            except RuntimeWarning:
                if debug:
                    print("\nlog10 error in the determination of SII_over_Halpha_Ratio_LinCon:\n"
                          "Looks like a zero got in there somewhere"
                          "\nSetting value to -1000 and continuing.")
                SII_over_Halpha_Ratio_LinCon = -1000
                Failed_SII_over_Halpha_Line_Ratio.append(ObjectName)

        if Spectral_Lines_For_Analysis_Air['[OI]6300'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Halpha'][16] in Exception_Values:
            OI_over_Halpha_Ratio_LinCon = -1000
        else:
            try:
                OI_over_Halpha_Ratio_LinCon = np.log10(
                    Spectral_Lines_For_Analysis_Air['[OI]6300'][16] / Spectral_Lines_For_Analysis_Air['Halpha'][16])
            except FloatingPointError:
                if debug:
                    print("\nlog10 error in the determination of OI_over_Halpha_Ratio_LinCon:\n"
                          "Likely cause is one or both lines measured in absorption."
                          "\nSetting value to -1000 and continuing.")
                OI_over_Halpha_Ratio_LinCon = -1000
                Failed_OI_over_Halpha_Line_Ratio.append(ObjectName)
            except RuntimeWarning:
                if debug:
                    print("\nlog10 error in the determination of OI_over_Halpha_Ratio_LinCon:\n"
                          "Looks like a zero got in there somewhere"
                          "\nSetting value to -1000 and continuing.")
                OI_over_Halpha_Ratio_LinCon = -1000
                Failed_OI_over_Halpha_Line_Ratio.append(ObjectName)

        if Spectral_Lines_For_Analysis_Air['[OIII]5007'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['[OIII]4959'][16] in Exception_Values:
            OIII5007_over_OIII4959_Ratio_LinCon = -1000
        else:
            try:
                OIII5007_over_OIII4959_Ratio_LinCon = np.log10(
                    Spectral_Lines_For_Analysis_Air['[OIII]5007'][16] / Spectral_Lines_For_Analysis_Air['[OIII]4959'][16])
            except FloatingPointError:
                if debug:
                    print("\nlog10 error in the determination of [OIII]5007_over_[OIII]4959_Ratio_LinCon:\n"
                          "Likely cause is one or both lines measured in absorption."
                          "\nSetting value to -1000 and continuing.")
                OIII5007_over_OIII4959_Ratio_LinCon = -1000
                Failed_OIII5007_over_OIII4959_Ratio.append(ObjectName)
            except RuntimeWarning:
                if debug:
                    print("\nlog10 error in the determination of [OIII]5007_over_[OIII]4959_Ratio_LinCon:\n"
                          "Looks like a zero got in there somewhere"
                          "\nSetting value to -1000 and continuing.")
                OIII5007_over_OIII4959_Ratio_LinCon = -1000
                Failed_OIII5007_over_OIII4959_Ratio.append(ObjectName)

        if Spectral_Lines_For_Analysis_Air['HeII4686'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Halpha'][16] in Exception_Values:
            HeII_over_Halpha_Ratio_LinCon = -1000
        else:
            try:
                HeII_over_Halpha_Ratio_LinCon = np.log10(
                    Spectral_Lines_For_Analysis_Air['HeII4686'][16] / Spectral_Lines_For_Analysis_Air['Halpha'][16])
            except FloatingPointError:
                if debug:
                    print("\nlog10 error in the determination of HeII_over_Halpha_LinCon:\n"
                          "Likely cause is one or both lines measured in absorption."
                          "\nSetting value to -1000 and continuing.")
                HeII_over_Halpha_Ratio_LinCon = -1000
                Failed_HeII_over_Halpha_Ratio.append(ObjectName)
            except RuntimeWarning:
                if debug:
                    print("\nlog10 error in the determination of lin_con_HeII_over_Halpha:\n"
                          "Looks like a zero got in there somewhere"
                          "\nSetting value to -1000 and continuing.")
                HeII_over_Halpha_Ratio_LinCon = -1000
                Failed_HeII_over_Halpha_Ratio.append(ObjectName)

        if HeII_over_Halpha_Ratio_LinCon == math.inf or NII_over_Halpha_Ratio_LinCon == -1 * math.inf:
            print(f"Looks like a cheeky zero still managed to squeak through, setting NII/Halpha to -1000 "
                  f"for {ObjectName}")
            HeII_over_Halpha_Ratio_LinCon = -1000
            Failed_NII_over_Halpha_Line_Ratio.append(ObjectName)

        if OIII_over_Hbeta_Ratio_LinCon == math.inf or OIII_over_Hbeta_Ratio_LinCon == -1 * math.inf:
            print(f"Looks like a cheeky zero still managed to squeak through, setting OIII/Hbeta to -1000 "
                  f"for {ObjectName}")
            OIII_over_Hbeta_Ratio_LinCon = -1000
            Failed_OIII_over_Hbeta_Line_Ratio.append(ObjectName)

        if SII_over_Halpha_Ratio_LinCon == math.inf or SII_over_Halpha_Ratio_LinCon == -1 * math.inf:
            print(f"Looks like a cheeky zero still managed to squeak through, setting SII/Halpha to -1000 "
                  f"for {ObjectName}")
            SII_over_Halpha_Ratio_LinCon = -1000
            Failed_SII_over_Halpha_Line_Ratio.append(ObjectName)

        if OI_over_Halpha_Ratio_LinCon == math.inf or OI_over_Halpha_Ratio_LinCon == -1 * math.inf:
            print(f"Looks like a cheeky zero still managed to squeak through, setting OI/Halpha to -1000 "
                  f"for {ObjectName}")
            OI_over_Halpha_Ratio_LinCon = -1000
            Failed_OI_over_Halpha_Line_Ratio.append(ObjectName)

        if OIII5007_over_OIII4959_Ratio_LinCon == math.inf or OIII5007_over_OIII4959_Ratio_LinCon == -1 * math.inf:
            print(f"Looks like a cheeky zero still managed to squeak through, setting OIII5007/OIII4959 to -1000 "
                  f"for {ObjectName}")
            OIII5007_over_OIII4959_Ratio_LinCon = -1000
            Failed_OIII5007_over_OIII4959_Ratio.append(ObjectName)

        if OIII5007_over_OIII4959_Ratio_LinCon == math.inf or OIII5007_over_OIII4959_Ratio_LinCon == -1 * math.inf:
            print(f"Looks like a cheeky zero still managed to squeak through, setting OIII5007/OIII4959 to -1000 "
                  f"for {ObjectName}")
            OIII5007_over_OIII4959_Ratio_LinCon = -1000
            Failed_OIII5007_over_OIII4959_Ratio.append(ObjectName)

        if debug:
            print("\n")
            print(ObjectName)
            print(OIII_over_Hbeta_Ratio_LinCon)
            print(NII_over_Halpha_Ratio_LinCon)
            print(SII_over_Halpha_Ratio_LinCon)
            print(OI_over_Halpha_Ratio_LinCon)
            print(OIII_over_Hbeta_Ratio_LinCon)
            print(Spectral_Lines_For_Analysis_Air['[NII]6584'][16])
            print(Spectral_Lines_For_Analysis_Air['Halpha'][16])
            print("\n")

        ##############################
        # Lick Indices Calculation
        ##############################

        try:
            Lick_Index_Data = Hirogen_Functions.lick_index_calculation(wave=wave.value, flux=flux.value, err=error)
        except:
            print("Lick Index Calculation Failure")
            Lick_Index_Data["d4000_n"] = -500
            Lick_Index_Data["d4000_n_err"] = -500
            Lick_Index_Data["lick_hd_a"] = -500
            Lick_Index_Data["lick_hg_a_err"] -500
            Failed_Lick.append(ObjectName)

        ##############################
        # NaN Screening
        ##############################

        for index in Lick_Index_Data:
            Lick_Index_Data[index] = -1000 if np.isnan(Lick_Index_Data[index]) else Lick_Index_Data[index]

        D4000 = Lick_Index_Data["d4000_n"]
        D4000_Err = Lick_Index_Data["d4000_n_err"]
        Lick_HDelta_Index = Lick_Index_Data["lick_hd_a"]
        Lick_HDelta_Index_Err = Lick_Index_Data["lick_hd_a_err"]
        Lick_HGamma_Index = Lick_Index_Data["lick_hg_a"]
        Lick_HGamma_Index_Err = Lick_Index_Data["lick_hg_a_err"]

        ##############################
        # Additional Object Flagging
        ##############################

        if Spectral_Lines_For_Analysis_Air['Halpha'][8] > -3: # Flags on the strength of weak Balmer emission
            Quiescent_Flag = 1
        else:
            Quiescent_Flag = 0

        if Lick_HDelta_Index == -1000:
            Balmer_Strong_Flag = -1
            Balmer_Moderately_Strong_Flag = -1
        else:

            if Lick_HDelta_Index - Lick_HDelta_Index_Err > 4:
                Balmer_Strong_Flag = 1
            else:
                Balmer_Strong_Flag = 0

            if Lick_HDelta_Index - Lick_HDelta_Index_Err > 1.31:
                Balmer_Moderately_Strong_Flag = 1
            else:
                Balmer_Moderately_Strong_Flag = 0


        ##############################
        # Database Update
        ##############################

        if not Local_DESI:

            # Again red 'errors' here are is just PyCharm not coping with the Table name as a variable - it will run fine
            cmd = f"update `{Database}`.`{TableID}`  " \
                  f"set lin_con_pEQW_Halpha = '{Spectral_Lines_For_Analysis_Air['Halpha'][8]}'," \
                  f" lin_con_pEQW_Hbeta = '{Spectral_Lines_For_Analysis_Air['Hbeta'][8]}'," \
                  f" lin_con_pEQW_Hgamma = '{Spectral_Lines_For_Analysis_Air['Hgamma'][8]}'," \
                  f" lin_con_pEQW_Hdelta = '{Spectral_Lines_For_Analysis_Air['Hdelta'][8]}'," \
                  f" lin_con_pEQW_NII6548 = '{Spectral_Lines_For_Analysis_Air['[NII]6548'][8]}'," \
                  f" lin_con_pEQW_NII6584 = '{Spectral_Lines_For_Analysis_Air['[NII]6584'][8]}'," \
                  f" lin_con_pEQW_FeVII3759 = '{Spectral_Lines_For_Analysis_Air['[FeVII]3759'][8]}'," \
                  f" lin_con_pEQW_FeVII5160 = '{Spectral_Lines_For_Analysis_Air['[FeVII]5160'][8]}'," \
                  f" lin_con_pEQW_FeVII5722 = '{Spectral_Lines_For_Analysis_Air['[FeVII]5722'][8]}'," \
                  f" lin_con_pEQW_FeVII6008 = '{Spectral_Lines_For_Analysis_Air['[FeVII]6088'][8]}'," \
                  f" lin_con_pEQW_FeX6376 = '{Spectral_Lines_For_Analysis_Air['[FeX]6376'][8]}'," \
                  f" lin_con_pEQW_FeXI7894 = '{Spectral_Lines_For_Analysis_Air['[FeXI]7894'][8]}'," \
                  f" lin_con_pEQW_FeXIV5304 = '{Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][8]}'," \
                  f" lin_con_pEQW_OI6300 = '{Spectral_Lines_For_Analysis_Air['[OI]6300'][8]}'," \
                  f" lin_con_pEQW_OI6363 = '{Spectral_Lines_For_Analysis_Air['[OI]6363'][8]}'," \
                  f" lin_con_pEQW_OII3728 = '{Spectral_Lines_For_Analysis_Air['[OII]3728'][8]}'," \
                  f" lin_con_pEQW_OIII4959 = '{Spectral_Lines_For_Analysis_Air['[OIII]4959'][8]}'," \
                  f" lin_con_pEQW_OIII5007 = '{Spectral_Lines_For_Analysis_Air['[OIII]5007'][8]}'," \
                  f" lin_con_pEQW_HeI4478 = '{Spectral_Lines_For_Analysis_Air['HeI4478'][8]}'," \
                  f" lin_con_pEQW_HeII4686 = '{Spectral_Lines_For_Analysis_Air['HeII4686'][8]}'," \
                  f" lin_con_pEQW_NaID = '{Spectral_Lines_For_Analysis_Air['NaID'][8]}'," \
                  f" lin_con_pEQW_SII6717 = '{Spectral_Lines_For_Analysis_Air['[SII]6716'][8]}'," \
                  f" lin_con_pEQW_SII6731 = '{Spectral_Lines_For_Analysis_Air['[SII]6731'][8]}'," \
                  f" lin_con_pEQW_SII6717_6731 = '{Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][8]}'," \
                  f" lin_con_pEQW_SIII6313 = '{Spectral_Lines_For_Analysis_Air['[SIII]6313'][8]}'," \
                  f" lin_con_LineFlux_Halpha = '{Spectral_Lines_For_Analysis_Air['Halpha'][16]}'," \
                  f" lin_con_LineFlux_Hbeta = '{Spectral_Lines_For_Analysis_Air['Hbeta'][16]}'," \
                  f" lin_con_LineFlux_Hgamma = '{Spectral_Lines_For_Analysis_Air['Hgamma'][16]}'," \
                  f" lin_con_LineFlux_Hdelta = '{Spectral_Lines_For_Analysis_Air['Hdelta'][16]}'," \
                  f" lin_con_LineFlux_NII6548 = '{Spectral_Lines_For_Analysis_Air['[NII]6548'][16]}'," \
                  f" lin_con_LineFlux_NII6584 = '{Spectral_Lines_For_Analysis_Air['[NII]6584'][16]}'," \
                  f" lin_con_LineFlux_FeVII3759 = '{Spectral_Lines_For_Analysis_Air['[FeVII]3759'][16]}'," \
                  f" lin_con_LineFlux_FeVII5160 = '{Spectral_Lines_For_Analysis_Air['[FeVII]5160'][16]}'," \
                  f" lin_con_LineFlux_FeVII5722 = '{Spectral_Lines_For_Analysis_Air['[FeVII]5722'][16]}'," \
                  f" lin_con_LineFlux_FeVII6008 = '{Spectral_Lines_For_Analysis_Air['[FeVII]6088'][16]}'," \
                  f" lin_con_LineFlux_FeX6376 = '{Spectral_Lines_For_Analysis_Air['[FeX]6376'][16]}'," \
                  f" lin_con_LineFlux_FeXI7894 = '{Spectral_Lines_For_Analysis_Air['[FeXI]7894'][16]}'," \
                  f" lin_con_LineFlux_FeXIV5304 = '{Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][16]}'," \
                  f" lin_con_LineFlux_OI6300 = '{Spectral_Lines_For_Analysis_Air['[OI]6300'][16]}'," \
                  f" lin_con_LineFlux_OI6363 = '{Spectral_Lines_For_Analysis_Air['[OI]6363'][16]}'," \
                  f" lin_con_LineFlux_OII3728 = '{Spectral_Lines_For_Analysis_Air['[OII]3728'][16]}'," \
                  f" lin_con_LineFlux_OIII4959 = '{Spectral_Lines_For_Analysis_Air['[OIII]4959'][16]}'," \
                  f" lin_con_LineFlux_OIII5007 = '{Spectral_Lines_For_Analysis_Air['[OIII]5007'][16]}'," \
                  f" lin_con_LineFlux_HeI4478 = '{Spectral_Lines_For_Analysis_Air['HeI4478'][16]}'," \
                  f" lin_con_LineFlux_HeII4686 = '{Spectral_Lines_For_Analysis_Air['HeII4686'][16]}'," \
                  f" lin_con_LineFlux_NaID = '{Spectral_Lines_For_Analysis_Air['NaID'][16]}'," \
                  f" lin_con_LineFlux_SII6717 = '{Spectral_Lines_For_Analysis_Air['[SII]6716'][16]}'," \
                  f" lin_con_LineFlux_SII6731 = '{Spectral_Lines_For_Analysis_Air['[SII]6731'][16]}'," \
                  f" lin_con_LineFlux_SII6717_6731 = '{Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][16]}'," \
                  f" lin_con_LineFlux_SIII6313 = '{Spectral_Lines_For_Analysis_Air['[SIII]6313'][16]}'," \
                  f" lin_con_NII_over_Halpha = '{NII_over_Halpha_Ratio_LinCon}'," \
                  f" lin_con_OIII_over_Hbeta = '{OIII_over_Hbeta_Ratio_LinCon}'," \
                  f" lin_con_SII_over_Halpha = '{SII_over_Halpha_Ratio_LinCon}'," \
                  f" lin_con_OI6300_over_Halpha = '{OI_over_Halpha_Ratio_LinCon}'," \
                  f" lin_con_OIII5007_over_OIII4959 = '{OIII5007_over_OIII4959_Ratio_LinCon}'," \
                  f" lin_con_HeII_over_Halpha = '{HeII_over_Halpha_Ratio_LinCon}'," \
                  f" Strong_FeVII_Flag = '{Strong_FeVII_Flag}'," \
                  f" Strong_FeX_Flag = '{Strong_FeX_Flag}'," \
                  f" Strong_FeXI_Flag = '{Strong_FeXI_Flag}'," \
                  f" Strong_FeXIV_Flag = '{Strong_FeXIV_Flag}'," \
                  f" ECLE_Candidate_Score = '{Candidate_Score}'," \
                  f" Pixels_With_Any_Flags_Percentage = '{All_Flags_Percentage}', " \
                  f" Pixels_With_Counted_Flags_Percentage = '{Counted_Flags_Percentage}', " \
                  f" D4000 = '{D4000}', " \
                  f" D4000_Err = '{D4000_Err}', " \
                  f" Lick_HGamma_Index = '{Lick_HGamma_Index}', " \
                  f" Lick_HGamma_Index_Err = '{Lick_HGamma_Index_Err}', " \
                  f" Lick_HDelta_Index = '{Lick_HDelta_Index}', " \
                  f" Lick_HDelta_Index_Err = '{Lick_HDelta_Index_Err}', " \
                  f" Quiescent_Flag = '{Quiescent_Flag}', " \
                  f" Balmer_Strong_Flag = '{Balmer_Strong_Flag}', " \
                  f" Balmer_Moderately_Strong_Flag = '{Balmer_Moderately_Strong_Flag}', " \
                  f" Settings_Config = '{Settings_Config}' " \
                  f"where DR16_Spectroscopic_ID = '{ObjectName}' "
            if debug:
                print(cmd)

            cursor.execute(cmd)
            Hirogen_Functions.commit(Data)

            Spectrum_Number += 1
            # Variable iteration

        else: # Local_DESI

             # Again red 'errors' here are is just PyCharm not coping with the Table name as a variable - it will run fine
            cmd = f"update `{Database}`.`{TableID}`  " \
                  f"set lin_con_pEQW_Halpha = '{Spectral_Lines_For_Analysis_Air['Halpha'][8]}'," \
                  f" lin_con_pEQW_Hbeta = '{Spectral_Lines_For_Analysis_Air['Hbeta'][8]}'," \
                  f" lin_con_pEQW_Hgamma = '{Spectral_Lines_For_Analysis_Air['Hgamma'][8]}'," \
                  f" lin_con_pEQW_Hdelta = '{Spectral_Lines_For_Analysis_Air['Hdelta'][8]}'," \
                  f" lin_con_pEQW_NII6548 = '{Spectral_Lines_For_Analysis_Air['[NII]6548'][8]}'," \
                  f" lin_con_pEQW_NII6584 = '{Spectral_Lines_For_Analysis_Air['[NII]6584'][8]}'," \
                  f" lin_con_pEQW_FeVII3759 = '{Spectral_Lines_For_Analysis_Air['[FeVII]3759'][8]}'," \
                  f" lin_con_pEQW_FeVII5160 = '{Spectral_Lines_For_Analysis_Air['[FeVII]5160'][8]}'," \
                  f" lin_con_pEQW_FeVII5722 = '{Spectral_Lines_For_Analysis_Air['[FeVII]5722'][8]}'," \
                  f" lin_con_pEQW_FeVII6008 = '{Spectral_Lines_For_Analysis_Air['[FeVII]6088'][8]}'," \
                  f" lin_con_pEQW_FeX6376 = '{Spectral_Lines_For_Analysis_Air['[FeX]6376'][8]}'," \
                  f" lin_con_pEQW_FeXI7894 = '{Spectral_Lines_For_Analysis_Air['[FeXI]7894'][8]}'," \
                  f" lin_con_pEQW_FeXIV5304 = '{Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][8]}'," \
                  f" lin_con_pEQW_OI6300 = '{Spectral_Lines_For_Analysis_Air['[OI]6300'][8]}'," \
                  f" lin_con_pEQW_OI6363 = '{Spectral_Lines_For_Analysis_Air['[OI]6363'][8]}'," \
                  f" lin_con_pEQW_OII3728 = '{Spectral_Lines_For_Analysis_Air['[OII]3728'][8]}'," \
                  f" lin_con_pEQW_OIII4959 = '{Spectral_Lines_For_Analysis_Air['[OIII]4959'][8]}'," \
                  f" lin_con_pEQW_OIII5007 = '{Spectral_Lines_For_Analysis_Air['[OIII]5007'][8]}'," \
                  f" lin_con_pEQW_HeI4478 = '{Spectral_Lines_For_Analysis_Air['HeI4478'][8]}'," \
                  f" lin_con_pEQW_HeII4686 = '{Spectral_Lines_For_Analysis_Air['HeII4686'][8]}'," \
                  f" lin_con_pEQW_NaID = '{Spectral_Lines_For_Analysis_Air['NaID'][8]}'," \
                  f" lin_con_pEQW_SII6717 = '{Spectral_Lines_For_Analysis_Air['[SII]6716'][8]}'," \
                  f" lin_con_pEQW_SII6731 = '{Spectral_Lines_For_Analysis_Air['[SII]6731'][8]}'," \
                  f" lin_con_pEQW_SII6717_6731 = '{Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][8]}'," \
                  f" lin_con_pEQW_SIII6313 = '{Spectral_Lines_For_Analysis_Air['[SIII]6313'][8]}'," \
                  f" lin_con_LineFlux_Halpha = '{Spectral_Lines_For_Analysis_Air['Halpha'][16]}'," \
                  f" lin_con_LineFlux_Hbeta = '{Spectral_Lines_For_Analysis_Air['Hbeta'][16]}'," \
                  f" lin_con_LineFlux_Hgamma = '{Spectral_Lines_For_Analysis_Air['Hgamma'][16]}'," \
                  f" lin_con_LineFlux_Hdelta = '{Spectral_Lines_For_Analysis_Air['Hdelta'][16]}'," \
                  f" lin_con_LineFlux_NII6548 = '{Spectral_Lines_For_Analysis_Air['[NII]6548'][16]}'," \
                  f" lin_con_LineFlux_NII6584 = '{Spectral_Lines_For_Analysis_Air['[NII]6584'][16]}'," \
                  f" lin_con_LineFlux_FeVII3759 = '{Spectral_Lines_For_Analysis_Air['[FeVII]3759'][16]}'," \
                  f" lin_con_LineFlux_FeVII5160 = '{Spectral_Lines_For_Analysis_Air['[FeVII]5160'][16]}'," \
                  f" lin_con_LineFlux_FeVII5722 = '{Spectral_Lines_For_Analysis_Air['[FeVII]5722'][16]}'," \
                  f" lin_con_LineFlux_FeVII6008 = '{Spectral_Lines_For_Analysis_Air['[FeVII]6088'][16]}'," \
                  f" lin_con_LineFlux_FeX6376 = '{Spectral_Lines_For_Analysis_Air['[FeX]6376'][16]}'," \
                  f" lin_con_LineFlux_FeXI7894 = '{Spectral_Lines_For_Analysis_Air['[FeXI]7894'][16]}'," \
                  f" lin_con_LineFlux_FeXIV5304 = '{Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][16]}'," \
                  f" lin_con_LineFlux_OI6300 = '{Spectral_Lines_For_Analysis_Air['[OI]6300'][16]}'," \
                  f" lin_con_LineFlux_OI6363 = '{Spectral_Lines_For_Analysis_Air['[OI]6363'][16]}'," \
                  f" lin_con_LineFlux_OII3728 = '{Spectral_Lines_For_Analysis_Air['[OII]3728'][16]}'," \
                  f" lin_con_LineFlux_OIII4959 = '{Spectral_Lines_For_Analysis_Air['[OIII]4959'][16]}'," \
                  f" lin_con_LineFlux_OIII5007 = '{Spectral_Lines_For_Analysis_Air['[OIII]5007'][16]}'," \
                  f" lin_con_LineFlux_HeI4478 = '{Spectral_Lines_For_Analysis_Air['HeI4478'][16]}'," \
                  f" lin_con_LineFlux_HeII4686 = '{Spectral_Lines_For_Analysis_Air['HeII4686'][16]}'," \
                  f" lin_con_LineFlux_NaID = '{Spectral_Lines_For_Analysis_Air['NaID'][16]}'," \
                  f" lin_con_LineFlux_SII6717 = '{Spectral_Lines_For_Analysis_Air['[SII]6716'][16]}'," \
                  f" lin_con_LineFlux_SII6731 = '{Spectral_Lines_For_Analysis_Air['[SII]6731'][16]}'," \
                  f" lin_con_LineFlux_SII6717_6731 = '{Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][16]}'," \
                  f" lin_con_LineFlux_SIII6313 = '{Spectral_Lines_For_Analysis_Air['[SIII]6313'][16]}'," \
                  f" lin_con_NII_over_Halpha = '{NII_over_Halpha_Ratio_LinCon}'," \
                  f" lin_con_OIII_over_Hbeta = '{OIII_over_Hbeta_Ratio_LinCon}'," \
                  f" lin_con_SII_over_Halpha = '{SII_over_Halpha_Ratio_LinCon}'," \
                  f" lin_con_OI6300_over_Halpha = '{OI_over_Halpha_Ratio_LinCon}'," \
                  f" lin_con_OIII5007_over_OIII4959 = '{OIII5007_over_OIII4959_Ratio_LinCon}'," \
                  f" lin_con_HeII_over_Halpha = '{HeII_over_Halpha_Ratio_LinCon}'," \
                  f" Strong_FeVII_Flag = '{Strong_FeVII_Flag}'," \
                  f" Strong_FeX_Flag = '{Strong_FeX_Flag}'," \
                  f" Strong_FeXI_Flag = '{Strong_FeXI_Flag}'," \
                  f" Strong_FeXIV_Flag = '{Strong_FeXIV_Flag}'," \
                  f" ECLE_Candidate_Score = '{Candidate_Score}'," \
                  f" Pixels_With_Any_Flags_Percentage = '{All_Flags_Percentage}', " \
                  f" Pixels_With_Counted_Flags_Percentage = '{Counted_Flags_Percentage}', " \
                  f" D4000 = '{D4000}', " \
                  f" D4000_Err = '{D4000_Err}', " \
                  f" Lick_HGamma_Index = '{Lick_HGamma_Index}', " \
                  f" Lick_HGamma_Index_Err = '{Lick_HGamma_Index_Err}', " \
                  f" Lick_HDelta_Index = '{Lick_HDelta_Index}', " \
                  f" Lick_HDelta_Index_Err = '{Lick_HDelta_Index_Err}', " \
                  f" Quiescent_Flag = '{Quiescent_Flag}', " \
                  f" Balmer_Strong_Flag = '{Balmer_Strong_Flag}', " \
                  f" Balmer_Moderately_Strong_Flag = '{Balmer_Moderately_Strong_Flag}', " \
                  f" Settings_Config = '{Settings_Config}' " \
                  f"where DESI_TARGET_ID = '{ObjectName}' "
            if debug:
                print(cmd)

            cursor.execute(cmd)
            Hirogen_Functions.commit(Data)

            Spectrum_Number += 1
            # Variable iteration


        #######
        # Candidate Filtering
        #######

        if Candidate_Score >= Candidate_Threshold or (Strong_FeVII_Flag+Strong_FeX_Flag+Strong_FeXI_Flag+Strong_FeXIV_Flag)> 0:
            Candidate = True
            Candidate_Count += 1
        else:
            Candidate = False

        ObjectCount += 1
        Successful_Objects_Count += 1


    # More disabled multi-threading code for the time being

    """
    comm.Barrier()

    comm.reduce(
        [ObjectCount, MPI.DOUBLE],
        op=MPI.SUM,
        root=0
    )

    if rank == 0:
        print(f"\n\nTotal number of spectra examined: {Total_Object_Count}")
        print(f"Total number of successful examinations: {Successful_Objects_Count}")
        print(f"Total number of failed examinations: {Failed_Objects_Count}")
        print(f"Total number of ECLE candidates: {Candidate_Count}")
        print(f"Total number of Strong [FeVII] Objects: {Strong_FeVII_Object_Count}")
        print(f"Total number of Strong [FeXIV] Objects: {Strong_FeXIV_Object_Count}")
        print(f"Failed NII/Halpha Ratio Objects: {Failed_NII_over_Halpha_Line_Ratio}")
        print(f"Failed OII/Hbeta Ratio Objects: {Failed_OIII_over_Hbeta_Line_Ratio}")
        print(f"Failed SII/Halpha Ratio Objects: {Failed_SII_over_Halpha_Line_Ratio}")
        print(f"Failed OI/Halpha Ratio Objects: {Failed_OI_over_Halpha_Line_Ratio}")

        endT = time.time()
        executionT = endT - startT

        print(f"Script execution time: {executionT:.2f}s")

    """

    print(f"Total number of spectra examined: {Total_Object_Count}")


    print(f"Total number of successful examinations: {Successful_Objects_Count}")
    print(f"Total number of failed examinations: {Failed_Objects_Count}")
    print(f"Total number of ECLE candidates: {Candidate_Count}")
    print(f"Total number of Strong [FeVII] Objects: {Strong_FeVII_Object_Count}")
    print(f"Total number of Strong [FeX] Objects: {Strong_FeX_Object_Count}")
    print(f"Total number of Strong [FeXI] Objects: {Strong_FeXI_Object_Count}")
    print(f"Total number of Strong [FeXIV] Objects: {Strong_FeXIV_Object_Count}")
    print(f"Failed NII/Halpha Ratio Objects: {Failed_NII_over_Halpha_Line_Ratio}")
    print(f"Failed OII/Hbeta Ratio Objects: {Failed_OIII_over_Hbeta_Line_Ratio}")
    print(f"Failed SII/Halpha Ratio Objects: {Failed_SII_over_Halpha_Line_Ratio}")
    print(f"Failed OI/Halpha Ratio Objects: {Failed_OI_over_Halpha_Line_Ratio}")
    print(f"Failed Lick Index Objects: {Failed_Lick}")

    endT = time.time()
    executionT = endT - startT

    print("Done")
    print(f"Script execution time: {executionT:.2f}s")

Now attempting to connect to the database: CoronalLineGalaxies
Connection Complete



./dr16/sdss/spectro/redux/26/spectra/0423/spec-0423-51821-0013-fake.fits
1500


  0%|          | 0/1500 [00:00<?, ?it/s]

476259264558557184
476262013337626624
476262563093440512
476263387727161344
476265311872509952
476265586750416896
476426390325979136
476426665203886080
476427764715513856
476428314471327744
476429413982955520
476430238616676352
476431338128304128
477383241528010752
477384066161731584
477385440551266304
477389838597777408
477390388353591296
477391487865219072
477393412010567680
477400009080334336
477400558836148224
477400833714055168
477401933225682944
477407155905914880
477407705661728768
477409629807077376
477409904684984320
477410179562891264
477410729318705152
477413752975681536
477416226876844032
477417326388471808
477418151022192640
477418425900099584
477419250533820416
477422274190796800
477423098824517632
477424473214052352
477424748091959296
477425297847773184
477428321504749568
477436292964050944
477439591498934272
477441240766375936
477442615155910656
477443439789631488
477447837836142592
477453610272188416
477470102946605056
477470652702418944
477471202458232832
477481098062

In [20]:
if NERSC_Flag:
    print('Running NERSC based analysis')
    
            # If operating on NERSC opens an ascii file to store the output results rather than accessing the database

    NERSC_Storage_File = open(f"Storage_File_for_BLOCK_ID_{BLOCK_ID}_BGS.ascii","w") # Hardcoded for BGS for the time being
    NERSC_Storage_File.write("#ID\tDate\tTile\tPetalNumber\tFiberID\tFiberStatus\tSpectrum_Number\tRA\tDEC\tRedshift\tRedshift_Err\tEBV\tpEQW_Halpha\tpEQW_Hbeta\tpEQW_Hgamma\tpEQW_Hdelta\tpEQW_NII6548\tpEQW_NII6584\tpEQW_FeVII3759\tpEQW_FeVII5160\tpEQW_FeVII5722\tpEQW_FeVII6088\tpEQW_FeX6376\tpEQW_FeXI7894\tpEQW_FeXIV5304\tpEQW_OI6300\tpEQW_OII3738\tpEQW_OIII4959\tpEQW_OIII5007\tpEQW_HeI4478\tpEQW_HeII4686\tpEQW_NaID\tpEQW_SII6716\tpEQW_SII6731\tpEQW_SII6716,6731\tLineFlux_Halpha\tLineFlux_Hbeta\tLineFlux_Hgamma\tLineFlux_Hdelta\tLineFlux_NII6548\tLineFlux_NII6584\tLineFlux_FeVII3759\tLineFlux_FeVII5160\tLineFlux_FeVII5722\tLineFlux_FeVII6088\tLineFlux_FeX6376\tLineFlux_FeXI7894\tLineFlux_FeXIV5304\tLineFlux_OI6300\tLineFlux_OII3738\tLineFlux_OIII4959\tLineFlux_OIII5007\tLineFlux_HeI4478\tLineFlux_HeII4686\tLineFlux_NaID\tLineFlux_SII6716\tLineFlux_SII6731\tLineFlux_SII6716,6731\tNII_over_Halpha\tOIII_over_Hbeta\tSII_over_Halpha\tOI_over_Halpha\tOIII5007_over_OIII4959\tHeII_over_Halpha\tD4000\tD4000_Err\tLick_HGamma_Index\tLick_HGamma_Index_Err\tLick_HDelta_Index\tLick_HDelta_Index_Err\tStrong_FeVII_Flag\tStrong_FeX_Flag\tStrong_FeXI_Flag\tStrong_FeXIV_Flag\tAll_Flags_Percentage\tCounted_Flags_Percentage\tQuiescent_Flag\tBalmer_Strong_Flag\tBalmer_Moderately_Strong_Flag\tScore\tGFiberFlux\tGFluxIvar\tRFiberFlux\tRFluxIvar\tZFiberFlux\tZerrFluxIvar\tW1Flux\tW1FluxIvar\tW2Flux\tW2FluxIvar\tIdentification_ID\tSettings_Config\n")

    Desired_Spectra_Filenames = []

    Spectral_Indices = []
    Spectral_Redshifts = []
    IDs = []
    RAs = []
    DECs = []
    EBVs = []
    Classifications = []
    Tiles = []

    # Loop through zbest and coadd files for each petal.
    # Extract the fibermaps, ZBEST tables, and spectra.
    # Keep only BGS targets passing basic event selection.
    allzbest = None
    allfmap = None
    allwave = None
    allflux = None
    allivar = None
    allmask = None
    allres  = None

    Number_of_Spectra_Per_Tile_List = []
    Number_of_Targets_Per_Tile_List = []
    Number_of_BGS_Targets_Per_Tile_List = []
    Number_of_Approved_Targets_Per_Tile_List = []

    Number_of_Spectra_Per_Petal = 0
    Number_of_Targets_Per_Petal = 0
    Number_of_BGS_Targets_Per_Tile = 0
    Number_of_Approved_Targets_Per_Petal = 0

    Total_Number_of_Spectra = 0
    Total_Number_of_Targets = 0
    Total_Number_of_BGS_Targets = 0
    Total_Number_of_Approved_Targets = 0

    # Per Block Directory
    PerBlock_SavePath = os.path.join(MainSavePath, f"Block_Data_for_Block_{BLOCK_ID}")
    Hirogen_Functions.directory_creation(PerBlock_SavePath)

    if Survey in ['EVEREST']:
        redux=f'/global/cfs/cdirs/desi/spectro/redux/everest/tiles/cumulative'

    else:
        redux='/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative'

    for tile in Tile_Numbers:
        Dates = [] # Store the date of the tile observations
        
        allzbest = None
        allfmap = None
        allwave = None
        allflux = None
        allivar = None
        allmask = None
        allres  = None

        prefix_in='/'.join([redux, '{}'.format(tile)])
        if not os.path.isdir(prefix_in):
            print('{} does not exist.'.format(prefix_in))
            continue

        else:
            subfolders = sorted(glob('{}/*'.format(prefix_in)))
            subfolder = max(subfolders) # This line restricts the file usage to the latest subfolder date
                                         # Given the short timespan between observations it is very unlikely any change would be seen in the cumulative spectra

        #for sf in subfolders: # If you want to run on ALL the different dates: uncomment this
            date = os.path.basename(subfolder) # If you want to run on ALL the different dates: swap subfolder to sf

            PerTile_SavePath = os.path.join(PerBlock_SavePath, f"Tile_{tile}")
            Hirogen_Functions.directory_creation(PerTile_SavePath)

            SummaryPlots_SavePath = os.path.join(PerTile_SavePath, f"Summary_Plots")
            Hirogen_Functions.directory_creation(SummaryPlots_SavePath)

            # Access the zbest and coadd files.
            # Files are organized by petal number.

            if Survey in ['EVEREST', 'MAIN']:
                zbfiles = sorted(glob('{}/redrock*.fits'.format(subfolder))) # If you want to run on ALL the different dates: swap subfolder to sf
            else:
                zbfiles = sorted(glob('{}/zbest*.fits'.format(subfolder))) # If you want to run on ALL the different dates: swap subfolder to sf

            cafiles = sorted(glob('{}/coadd*.fits'.format(subfolder))) # If you want to run on ALL the different dates: swap subfolder to sf

            print('Tile: {} - {}'.format(tile, date))

            Total_Number_of_Spectra_Per_Tile = 0
            Total_Number_of_Targets_Per_Tile = 0
            Total_Number_of_BGS_Targets_Per_Tile = 0
            Total_Number_of_Approved_Targets_Per_Tile = 0

            for cafile, zbfile in match_files(cafiles, zbfiles):

                petal = get_petal_id(cafile)

                print('  - Petal {}'.format(petal))

                # Access data per petal.
                if Survey in ['EVEREST', 'MAIN']:
                    zbest = Table.read(zbfile, 'REDSHIFTS')
                else:
                    zbest = Table.read(zbfile, 'ZBEST')

                pspectra = read_spectra(cafile)
                cspectra = coadd_cameras(pspectra)
                fibermap = cspectra.fibermap

                #All Object Coords
                All_RA = fibermap["TARGET_RA"]
                All_DEC = fibermap["TARGET_DEC"]

                if Survey == 'SV1':
                    isBGS = fibermap['SV1_BGS_TARGET'] & bgs_mask.mask(sv1_bgs_bits) != 0
                elif Survey == 'SV3':
                    isBGS = fibermap['SV3_BGS_TARGET'] & bgs_mask.mask(sv3_bgs_bits) != 0
                else:
                    bgsmask = desi_mask.mask('BGS_ANY')
                    desi_target = fibermap['DESI_TARGET']
                    isBGS = (desi_target & bgsmask) != 0

                # Apply standard event selection.
                isTGT = fibermap['OBJTYPE'] == 'TGT'
                isGAL = zbest['SPECTYPE'] == 'GALAXY'
                inZRange = zbest['Z'] <= 0.45 # A basic cut for now to filter off high z targets

                if Survey in ['MAIN', 'EVEREST']:
                    isGoodFiber = fibermap['COADD_FIBERSTATUS'] == 0
                else:
                    isGoodFiber = fibermap['FIBERSTATUS'] == 0

                isGoodZbest = (zbest['DELTACHI2'] > 25.) & (zbest['ZWARN'] == 0)
                select = isTGT & isGAL & isBGS & isGoodFiber & isGoodZbest

                print('     + selected: {}'.format(np.sum(select)))

                #Object Counts

                Number_of_Spectra_Per_Petal = len(fibermap["TARGETID"])
                Number_of_Targets_Per_Petal = len(fibermap["TARGETID"][isTGT])
                Number_of_BGS_Targets_Per_Petal = len(fibermap["TARGETID"][isBGS])
                Number_of_Approved_Targets_Per_Petal = len(fibermap["TARGETID"][select])

                Total_Number_of_Spectra_Per_Tile = Total_Number_of_Spectra_Per_Tile + Number_of_Spectra_Per_Petal
                Total_Number_of_Targets_Per_Tile = Total_Number_of_Targets_Per_Tile + Number_of_Targets_Per_Petal
                Total_Number_of_BGS_Targets_Per_Tile = Total_Number_of_BGS_Targets_Per_Tile + Number_of_BGS_Targets_Per_Petal
                Total_Number_of_Approved_Targets_Per_Tile = Total_Number_of_Approved_Targets_Per_Tile + Number_of_Approved_Targets_Per_Petal

                Total_Number_of_Spectra += Number_of_Spectra_Per_Petal
                Total_Number_of_Targets += Number_of_Targets_Per_Petal
                Total_Number_of_BGS_Targets += Number_of_BGS_Targets_Per_Petal
                Total_Number_of_Approved_Targets += Number_of_Approved_Targets_Per_Petal


                #Target Object Coords
                All_BGS_RA = fibermap["TARGET_RA"][isBGS]
                All_BGS_DEC = fibermap["TARGET_DEC"][isBGS]

                Target_RAs = fibermap["TARGET_RA"][select]
                Target_DECs = fibermap["TARGET_DEC"][select]

                Target_Display_Figure = plt.figure()
                plt.scatter(All_RA,All_DEC, color = 'k', marker = 'o', s = 15, label = 'Object', alpha= 0.2)
                plt.scatter(All_BGS_RA,All_BGS_DEC, color = 'r', marker = 'o', s = 10, label = 'BGS Target')
                plt.scatter(Target_RAs,Target_DECs, color = 'royalblue', marker = 'o', s = 5, label = 'Selected BGS Target')
                plt.legend()
                plt.xlabel('RA')
                plt.ylabel('DEC')
                plt.savefig(f"{SummaryPlots_SavePath}/SpecMap_for_Spectrograph_{petal}_Tile_{tile}_Date_{date}.pdf", dpi=200,
                                          bbox_inches='tight')
                plt.close()

                # Accumulate spectrum data.
                if allzbest is None:
                    allzbest = zbest[select]
                    allfmap = fibermap[select]
                    allwave = cspectra.wave['brz']
                    allflux = cspectra.flux['brz'][select]
                    allivar = cspectra.ivar['brz'][select]
                    allmask = cspectra.mask['brz'][select]
                    allres  = cspectra.resolution_data['brz'][select]
                else:
                    allzbest = vstack([allzbest, zbest[select]])
                    allfmap = vstack([allfmap, fibermap[select]])
                    allflux = np.vstack([allflux, cspectra.flux['brz'][select]])
                    allivar = np.vstack([allivar, cspectra.ivar['brz'][select]])
                    allmask = np.vstack([allmask, cspectra.mask['brz'][select]])
                    allres  = np.vstack([allres, cspectra.resolution_data['brz'][select]])

            jj = 0
            while jj < Total_Number_of_Approved_Targets_Per_Tile:
                Dates.append(date)
                jj += 1

            Number_of_Spectra_Per_Tile_List.append(Total_Number_of_Spectra_Per_Tile)
            Number_of_Targets_Per_Tile_List.append(Total_Number_of_Targets_Per_Tile)
            Number_of_BGS_Targets_Per_Tile_List.append(Total_Number_of_BGS_Targets_Per_Tile)
            Number_of_Approved_Targets_Per_Tile_List.append(Total_Number_of_Approved_Targets_Per_Tile)

        try:
            Total_Object_Count = len(allflux)
        except TypeError:
            Total_Object_Count = 0

        if Total_Object_Count == 0:
            print("\n\nNo data for this date / block / configuration")
            sys.exit()

        else:

            print(f"\n\nTile looked at: {tile}\nTotal number of spectra to be explored: {Total_Object_Count}")

            print(f"{Total_Number_of_Spectra}")
            print(f"{Total_Number_of_Targets}")
            print(f"{Total_Number_of_BGS_Targets}")

            print("Per tile lists:")
            print(f"{Tile_Numbers}")
            print(f"{Number_of_Spectra_Per_Tile_List}")
            print(f"{Number_of_Targets_Per_Tile_List}")
            print(f"{Number_of_BGS_Targets_Per_Tile_List}")
            print(f"{Number_of_Approved_Targets_Per_Tile_List}")

            print(f"\n\nFinished determining which spectra are desired obtained in Block ID: {BLOCK_ID}.")


            if Pickle_Dump: # Currently non-functional
                Pickle_Dump_Path = f'{MainPicklePath}/{BLOCK_ID}_BGS_Targets.pickle'
                Hirogen_DESI_Functions.desi_pickle_save(pickle_path = Pickle_Dump_Path, fibermap=allfmap, zbestdata=allzbest,
                                                        wavedata=allwave, fluxdata=allflux, ivardata=allivar,
                                                        dates=Dates, perdate_data=PerDateSpectral_Data, survey = Survey)

        # This retrieves the stored information from a Pickle File if needed, otherwise, i.e. flag not set just uses the data directly from concurrent analysis

        Pickle_Retrieval = False

        if Pickle_Retrieval: # Currently non-functional
            Pickle_Retrieval_Path = f'{MainPicklePath}/{date}_BGS_Targets.pickle'
            Retrieved_Data = Hirogen_DESI_Functions.desi_pickle_retriever(path=Pickle_Retrieval_Path)

            Object_Name_List = Retrieved_Data[0]
            """
            Retrieved_Ids = Retrieved_Data[0]
            Retrieved_Date = Retrieved_Data[1]
            Retrieved_Tile_Ids = Retrieved_Data[2]
            Retrieved_Spectrograph_Ids = Retrieved_Data[3]
            Retrieved_Fiber_Ids = Retrieved_Data[4]
            Retrieved_Fiberstatus = Retrieved_Data[5]
            Retrieved_Device_Locs = Retrieved_Data[6]
            Retrieved_RAs = Retrieved_Data[7]
            Retrieved_Decs = Retrieved_Data[8]
            Retrieved_EBVs = Retrieved_Data[9]
            Retrieved_REDSHIFT = Retrieved_Data[10]
            Retrieved_REDSHIFT_Err = Retrieved_Data[11]

            Retrieved_G = Retrieved_Data[12]
            Retrieved_G_Err = Retrieved_Data[13]
            Retrieved_R = Retrieved_Data[14]
            Retrieved_R_Err = Retrieved_Data[15]
            Retrieved_Z = Retrieved_Data[16]
            Retrieved_Z_Err = Retrieved_Data[17]

            Retrieved_W1 = Retrieved_Data[18]
            Retrieved_W2 = Retrieved_Data[19]

            Retrieved_Wave = Retrieved_Data[20]
            Retrieved_Flux = Retrieved_Data[21]
            Retrieved_Flux_Err = Retrieved_Data[22]
            """

            """
            print(Retrieved_Ids)
            print(Retrieved_Date)
            print(Retrieved_Spectrograph_Ids)
            print(Retrieved_Tile_Ids)
            print(Retrieved_Fiber_Ids)
            print(Retrieved_Device_Locs)
            print(Retrieved_RAs)
            print(Retrieved_Decs)
            print(Retrieved_EBVs)
            print(Retrieved_REDSHIFT)
            print(Retrieved_REDSHIFT_Err)
            print(Retrieved_G)
            print(Retrieved_G_Err)
            print(Retrieved_R)
            print(Retrieved_R_Err)
            print(Retrieved_Z)
            print(Retrieved_Z_Err)
            print(Retrieved_W1)
            print(Retrieved_W2)
            print(Retrieved_Wave)
            print(Retrieved_Flux)
            print(Retrieved_Flux_Err)
            """
        z_Correction_Override_List = [np.float64(0)] * Total_Object_Count
        Extinction_Correction_Override_List = [np.float64(0)] * Total_Object_Count

        print(Total_Object_Count)

        #################################
        # Counting Variables
        #################################

        Spectrum_Number = 0
        Successful_Objects_Count = 0
        Failed_Objects_Count = 0
        Candidate_Count = 0
        ObjectCount = 0
        Strong_FeVII_Object_Count = 0
        Strong_FeX_Object_Count = 0
        Strong_FeXI_Object_Count = 0
        Strong_FeXIV_Object_Count = 0

        Failed_NII_over_Halpha_Line_Ratio = []
        Failed_OIII_over_Hbeta_Line_Ratio = []
        Failed_SII_over_Halpha_Line_Ratio = []
        Failed_OI_over_Halpha_Line_Ratio = []
        Failed_OIII5007_over_OIII4959_Ratio = []
        Failed_HeII_over_Halpha_Ratio = []
        Failed_Lick = []

        for Spectrum_Number in tqdm(range(Total_Object_Count)):

            Candidate_Score = 0
            Strong_FeVII_Flag = 0
            Strong_FeX_Flag = 0
            Strong_FeXI_Flag = 0
            Strong_FeXIV_Flag = 0


            if not Pickle_Retrieval:

                if Spectrum_Number == 0:
                    Wavelength_Org = allwave

                ObjectName = f"{allfmap['TARGETID'][Spectrum_Number]}"
                #Date = f"{allfmap['LAST_NIGHT'][Spectrum_Number]}" # This doesn't seem to match the folder name exactly sometimes
                Date = Dates[Spectrum_Number]
                Redshift = allzbest['Z'][Spectrum_Number]
                Redshift_Err = allzbest['ZERR'][Spectrum_Number]
                Extinction = allfmap['EBV'][Spectrum_Number]
                Tile = allfmap['TILEID'][Spectrum_Number]
                Spectrograph = allfmap['PETAL_LOC'][Spectrum_Number]
                Fiber = allfmap['FIBER'][Spectrum_Number]

                if Survey in ['MAIN', 'EVEREST']:
                        FiberStatus = allfmap['COADD_FIBERSTATUS'][Spectrum_Number]
                else:
                        FiberStatus = allfmap['FIBERSTATUS'][Spectrum_Number]


                G_FiberFlux = allfmap['FIBERFLUX_G'][Spectrum_Number]
                G_Flux_Ivar = allfmap['FLUX_IVAR_G'][Spectrum_Number]

                R_FiberFlux = allfmap['FIBERFLUX_R'][Spectrum_Number]
                R_Flux_Ivar = allfmap['FLUX_IVAR_R'][Spectrum_Number]

                Z_FiberFlux = allfmap['FIBERFLUX_Z'][Spectrum_Number]
                Z_Flux_Ivar = allfmap['FLUX_IVAR_Z'][Spectrum_Number]

                W1_Flux = allfmap['FLUX_W1'][Spectrum_Number]

                try:
                    W1_Flux_Ivar = allfmap['FLUX_IVAR_W1'][Spectrum_Number]
                    if allfmap['FLUX_IVAR_W1'][Spectrum_Number] == '--':
                        W1_Flux_Ivar[ii] = -995
                except KeyError:
                    W1_Flux_Ivar = -995

                W2_Flux = allfmap['FLUX_W2'][Spectrum_Number]

                try:
                    W2_Flux_Ivar = allfmap['FLUX_IVAR_W2'][Spectrum_Number]
                    if allfmap['FLUX_IVAR_W2'][Spectrum_Number] == '--':
                        W2_Flux_Ivar[ii] = -995
                except KeyError:
                    W2_Flux_Ivar = -995


                RA = allfmap['TARGET_RA'][Spectrum_Number]
                DEC = allfmap['TARGET_DEC'][Spectrum_Number]

                Flux_Org = allflux[Spectrum_Number]
                Error_Org = allivar[Spectrum_Number]

            else:
                if Spectrum_Number == 0:
                    Wavelength_Org = Retrieved_Data[20]

                Flux_Org = Retrieved_Data[21][Spectrum_Number] # For the time being the actual spectra are also stored within the pickle files, depending on the amount of spectra this may change
                Error_Org = Retrieved_Data[22][Spectrum_Number]

                Redshift = Retrieved_Data[10][Spectrum_Number]
                Redshift_Err = Retrieved_Data[11][Spectrum_Number]
                Extinction = Retrieved_Data[9][Spectrum_Number]
                Tile = Retrieved_Data[2][Spectrum_Number]
                Spectrograph = Retrieved_Data[3][Spectrum_Number]
                Fiber = Retrieved_Data[4][Spectrum_Number]
                FiberStatus = Retrieved_Data[5][Spectrum_Number]

                ObjectName = f"{Retrieved_Data[0][Spectrum_Number]}"
                Date = Retrieved_Data[1][Spectrum_Number]
                RA = Retrieved_Data[7][Spectrum_Number]
                DEC = Retrieved_Data[8][Spectrum_Number]

                G_FiberFlux = Retrieved_Data[12][Spectrum_Number]
                G_Flux_Ivar = Retrieved_Data[13][Spectrum_Number]

                R_FiberFlux = Retrieved_Data[14][Spectrum_Number]
                R_Flux_Ivar = Retrieved_Data[15][Spectrum_Number]

                Z_FiberFlux = Retrieved_Data[16][Spectrum_Number]
                Z_Flux_Ivar = Retrieved_Data[17][Spectrum_Number]

                W1_Flux = Retrieved_Data[18][Spectrum_Number]
                W1_Flux_Ivar = Retrieved_Data[19][Spectrum_Number]

                W2_Flux = Retrieved_Data[20][Spectrum_Number]
                W2_Flux_Ivar = Retrieved_Data[21][Spectrum_Number]

            Identification_ID = f'{ObjectName}_{Tile}_{Spectrograph}_{Fiber}'
            flux = Flux_Org * u.Unit('erg cm-2 s-1 AA-1')  # Flux
            wave = Wavelength_Org * u.AA
            error = Error_Org

            #################################
            # Flag Exclusion will go here - DESI Objects are screened for inclusion generally beforehand using the DESItrip filters
            #################################

            All_Flags_Percentage = -999
            Counted_Flags_Percentage = -999


            #################################
            # Extinction Correction
            #################################

            if Extinction_Correction_Override_List[Spectrum_Number] == 0:
                extinction_corrected_flux = Hirogen_Functions.spectral_extinction_correction(flux, wave, extinction = Extinction)
                flux = extinction_corrected_flux

            #################################
            # Redshift Correction
            #################################
            if z_Correction_Override_List[Spectrum_Number] == 0:
                wave_rest = Hirogen_Functions.rest_wavelength_converter(observer_frame_wave=wave.value, z=Redshift)
                wave = wave_rest * u.AA

            ###########
            # Rebinning
            ###########

            if Spectres_Rebin:
                spectres_data = Hirogen_Functions.spectres_rebin(flux=flux.value, wave=wave.value, desired_res=Rebin_Res)
                flux = spectres_data[0] * u.Unit('erg cm-2 s-1 AA-1')
                wave = spectres_data[1] * u.Unit('AA')

            ###########
            # Smoothing
            ###########

            if Smoothing:  # If active runs a boxcar smooth, defaults to a width of 5 but can be set via the function call
                smoothed_flux = Hirogen_Functions.spectral_flux_smooth(flux)
                flux = smoothed_flux

            ###########
            # Other, non-boxcar Smoothing
            ###########

            if FFT_Denoise:
                filtered_flux = Hirogen_Functions.fft_filter_signal(flux.value, threshold=FFT_Threshold)
                flux = filtered_flux * u.Unit('erg cm-2 s-1 AA-1')

            if Median_Filter: # Runs a median filter, defaults to a kernel of size 3 but can be set via the function call
                med_filtered_flux = Hirogen_Functions.spectral_median_filter(flux, med_filter_kernel= Median_Filter_Kernel)
                flux = med_filtered_flux * u.Unit('erg cm-2 s-1 AA-1')

            if Gaussian_Smooth:
                gaussian_smoothed_flux = Hirogen_Functions.gaussian_smoothing(flux.value, sigma_value = Gaussian_Sigma)
                flux = gaussian_smoothed_flux * u.Unit('erg cm-2 s-1 AA-1')

            ######
            # General check and removal of -ve flux points
            ######

            """
            for ii, item in enumerate(flux):
                if flux[ii].value < 0:
                    flux[ii] = np.nan * u.Unit('erg cm-2 s-1 AA-1')
            """

            #######
            # Spectral readin + processing for one spectrum completed
            #######

            ###########
            # Line pEQW measurements with linear continuum subtraction
            ###########

            for ii, item in enumerate(Spectral_Lines_For_Analysis_Air):

                Line_Location = Spectral_Lines_For_Analysis_Air[item][0]

                # Velocity shift from line
                Shift = (((np.array(wave) * c) / Line_Location) - c)

                # Cut down the region for analysis
                # This may need to be tweaked currently 75 angstroms either side of the line
                # Will default to nans if spectrum does not extend to fully include these regions

                if wave[-1].value < Line_Location - 75 or wave[1].value > Line_Location + 75:

                    if debug:
                        print(f"Analysis region \n{Spectral_Lines_For_Analysis_Air_Names[ii]} "
                              f"is not fully covered in the rest frame.")

                    Spectral_Lines_For_Analysis_Air[item][2] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][3] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][4] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][5] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][6] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][7] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][8] = -999
                    Spectral_Lines_For_Analysis_Air[item][9] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][10] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][11] = np.nan
                    Spectral_Lines_For_Analysis_Air[item][16] = -999

                else:
                    Shift_Region, Wave_Region, Flux_Region, Error_Region = Hirogen_Functions.region_cutter(
                        shift=Shift, wave=wave,
                        flux=flux,
                        low_cut=Line_Location - Lower_Wave,
                        high_cut=Line_Location + Upper_Wave,
                        mode='Wavelength'
                    )

                    # Generate and remove a continuum fit from the flux data - currently linear other functions seem worse
                    # The scaled flux output - Continuum[1] - is scaled to a continuum value of 1
                    # The returned continuum values - Continuum[0] - is relative to the original unscaled spectrum
                    Continuum = Hirogen_Functions.continua_maker(
                        spec_region=Wave_Region,
                        flux=Flux_Region,
                        # shift=Shift_Region,
                        line_name=list(Spectral_Lines_For_Analysis_Air)[ii],
                        line_loc=Line_Location,
                        object_name=ObjectName
                    )

                    # Measure the resulting pEQW of the feature

                    pEQW_LinCon_Spec, Line_Flux = Hirogen_Functions.eqw_measurement(
                        flux = Flux_Region,
                        true_continuum = Continuum[0],
                        scaled_flux=Continuum[1],
                        xaxis=Wave_Region,
                        line_loc=Line_Location
                    )

                    # Store the results in the main dictionary

                    Spectral_Lines_For_Analysis_Air[item][2] = Shift
                    Spectral_Lines_For_Analysis_Air[item][3] = Shift_Region
                    Spectral_Lines_For_Analysis_Air[item][4] = Wave_Region
                    Spectral_Lines_For_Analysis_Air[item][5] = Flux_Region
                    Spectral_Lines_For_Analysis_Air[item][6] = Continuum[0]
                    Spectral_Lines_For_Analysis_Air[item][7] = Continuum[1]

                    if np.isnan(pEQW_LinCon_Spec):
                        Spectral_Lines_For_Analysis_Air[item][8] = -9999
                    else:
                        Spectral_Lines_For_Analysis_Air[item][8] = pEQW_LinCon_Spec

                    if np.isnan(Line_Flux):
                        Spectral_Lines_For_Analysis_Air[item][16] = -9999
                    else:
                        Spectral_Lines_For_Analysis_Air[item][16] = Line_Flux

                    ##############################
                    # Candidate Conditions Checks
                    ##############################

                    if Spectral_Lines_For_Analysis_Air_Names[ii] in Spectral_Lines_For_Scoring_Air:

                        LineScore = Hirogen_Functions.lin_con_candidate_line_identifier(
                            line_name=Spectral_Lines_For_Analysis_Air_Names[ii],
                            line_peqw=Spectral_Lines_For_Analysis_Air[item][8],
                            peqw_threshold=LineDetection_pEQW_Threshold,
                            continuum_removed_flux=Continuum[1],
                            shift_points=Shift_Region,
                            peak_threshold=LineDetection_Max_Threshold,
                            above_continuum_threshold=LineDetection_Max_Above_Av_Continua_Threshold,
                            peak_max_region=LineDetection_Peak_Tolerance,
                            peak_region=Line_Peak_Location_Region,
                            feature_region=3000,
                            peak_region_minima_threshold=Line_Peak_Region_Minima_Threshold
                        )
                        Candidate_Score = Candidate_Score + LineScore[0]
                        Spectral_Lines_For_Analysis_Air[item][9] = LineScore[4]
                        Spectral_Lines_For_Analysis_Air[item][10] = LineScore[2]
                        Spectral_Lines_For_Analysis_Air[item][11] = LineScore[7]
                    else:
                        Spectral_Lines_For_Analysis_Air[item][9] = np.nan
                        Spectral_Lines_For_Analysis_Air[item][10] = np.nan
                        Spectral_Lines_For_Analysis_Air[item][11] = np.nan

            if Spectral_Lines_For_Analysis_Air['[FeVII]6088'][8] < Strong_EQW_Threshold and \
                    Spectral_Lines_For_Analysis_Air['[FeVII]6088'][9] > Strong_Peak_Max_Threshold:
                Strong_FeVII_Flag = 1
                Strong_FeVII_Object_Count += 1

            if Spectral_Lines_For_Analysis_Air['[FeX]6376'][8] < Strong_EQW_Threshold and \
                    Spectral_Lines_For_Analysis_Air['[FeX]6376'][9] > Strong_Peak_Max_Threshold:
                Strong_FeX_Flag = 1
                Strong_FeX_Object_Count += 1

            if Spectral_Lines_For_Analysis_Air['[FeXI]7894'][8] < Strong_EQW_Threshold and \
                    Spectral_Lines_For_Analysis_Air['[FeXI]7894'][9] > Strong_Peak_Max_Threshold:
                Strong_FeXI_Flag = 1
                Strong_FeXI_Object_Count += 1

            if Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][8] < Strong_EQW_Threshold and \
                    Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][9] > Strong_Peak_Max_Threshold:
                Strong_FeXIV_Flag = 1
                Strong_FeXIV_Object_Count += 1

            if Spectral_Lines_For_Analysis_Air['[NII]6584'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Halpha'][16] in Exception_Values:
                NII_over_Halpha_Ratio_LinCon = -1000
            else:
                try:
                    NII_over_Halpha_Ratio_LinCon = np.log10(
                        Spectral_Lines_For_Analysis_Air['[NII]6584'][16] / Spectral_Lines_For_Analysis_Air['Halpha'][16])
                except FloatingPointError:
                    if debug:
                        print("\nlog10 error in the determination of NII_over_Halpha_Ratio_LinCon:\n"
                              "Likely cause is one or both lines measured in absorption."
                              "\nSetting value to -1000 and continuing.")
                    NII_over_Halpha_Ratio_LinCon = -1000
                    Failed_NII_over_Halpha_Line_Ratio.append(ObjectName)
                except RuntimeWarning:
                    if debug:
                        print("\nlog10 error in the determination of NII_over_Halpha_Ratio_LinCon:\n"
                              "Looks like a zero got in there somewhere"
                              "\nSetting value to -1000 and continuing.")
                    NII_over_Halpha_Ratio_LinCon = -1000
                    Failed_NII_over_Halpha_Line_Ratio.append(ObjectName)

            if Spectral_Lines_For_Analysis_Air['[OIII]5007'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Hbeta'][16] in Exception_Values:
                OIII_over_Hbeta_Ratio_LinCon = -1000
            else:
                try:
                    OIII_over_Hbeta_Ratio_LinCon = np.log10(
                        Spectral_Lines_For_Analysis_Air['[OIII]5007'][16] / Spectral_Lines_For_Analysis_Air['Hbeta'][16])
                except FloatingPointError:
                    if debug:
                        print("\nlog10 error in the determination of OIII_over_Hbeta_Ratio_LinCon:\n"
                              "Likely cause is one or both lines measured in absorption."
                              "\nSetting value to -1000 and continuing.")
                    OIII_over_Hbeta_Ratio_LinCon = -1000
                    Failed_OIII_over_Hbeta_Line_Ratio.append(ObjectName)
                except RuntimeWarning:
                    if debug:
                        print("\nlog10 error in the determination of OIII_over_Hbeta_Ratio_LinCon:\n"
                              "Looks like a zero got in there somewhere"
                              "\nSetting value to -1000 and continuing.")
                    OIII_over_Hbeta_Ratio_LinCon = -1000
                    Failed_OIII_over_Hbeta_Line_Ratio.append(ObjectName)

            if Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Halpha'][16] in Exception_Values:
                SII_over_Halpha_Ratio_LinCon = -1000
            else:
                try:
                    SII_over_Halpha_Ratio_LinCon = np.log10(
                        Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][16] / Spectral_Lines_For_Analysis_Air['Halpha'][16])
                except FloatingPointError:
                    if debug:
                        print("\nlog10 error in the determination of SII_over_Halpha_Ratio_LinCon:\n"
                              "Likely cause is one or both lines measured in absorption."
                              "\nSetting value to -1000 and continuing.")
                    SII_over_Halpha_Ratio_LinCon = -1000
                    Failed_SII_over_Halpha_Line_Ratio.append(ObjectName)
                except RuntimeWarning:
                    if debug:
                        print("\nlog10 error in the determination of SII_over_Halpha_Ratio_LinCon:\n"
                              "Looks like a zero got in there somewhere"
                              "\nSetting value to -1000 and continuing.")
                    SII_over_Halpha_Ratio_LinCon = -1000
                    Failed_SII_over_Halpha_Line_Ratio.append(ObjectName)

            if Spectral_Lines_For_Analysis_Air['[OI]6300'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Halpha'][16] in Exception_Values:
                OI_over_Halpha_Ratio_LinCon = -1000
            else:
                try:
                    OI_over_Halpha_Ratio_LinCon = np.log10(
                        Spectral_Lines_For_Analysis_Air['[OI]6300'][16] / Spectral_Lines_For_Analysis_Air['Halpha'][16])
                except FloatingPointError:
                    if debug:
                        print("\nlog10 error in the determination of OI_over_Halpha_Ratio_LinCon:\n"
                              "Likely cause is one or both lines measured in absorption."
                              "\nSetting value to -1000 and continuing.")
                    OI_over_Halpha_Ratio_LinCon = -1000
                    Failed_OI_over_Halpha_Line_Ratio.append(ObjectName)
                except RuntimeWarning:
                    if debug:
                        print("\nlog10 error in the determination of OI_over_Halpha_Ratio_LinCon:\n"
                              "Looks like a zero got in there somewhere"
                              "\nSetting value to -1000 and continuing.")
                    OI_over_Halpha_Ratio_LinCon = -1000
                    Failed_OI_over_Halpha_Line_Ratio.append(ObjectName)

            if Spectral_Lines_For_Analysis_Air['[OIII]5007'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['[OIII]4959'][16] in Exception_Values:
                OIII5007_over_OIII4959_Ratio_LinCon = -1000
            else:
                try:
                    OIII5007_over_OIII4959_Ratio_LinCon = np.log10(
                        Spectral_Lines_For_Analysis_Air['[OIII]5007'][16] / Spectral_Lines_For_Analysis_Air['[OIII]4959'][16])
                except FloatingPointError:
                    if debug:
                        print("\nlog10 error in the determination of [OIII]5007_over_[OIII]4959_Ratio_LinCon:\n"
                              "Likely cause is one or both lines measured in absorption."
                              "\nSetting value to -1000 and continuing.")
                    OIII5007_over_OIII4959_Ratio_LinCon = -1000
                    Failed_OIII5007_over_OIII4959_Ratio.append(ObjectName)
                except RuntimeWarning:
                    if debug:
                        print("\nlog10 error in the determination of [OIII]5007_over_[OIII]4959_Ratio_LinCon:\n"
                              "Looks like a zero got in there somewhere"
                              "\nSetting value to -1000 and continuing.")
                    OIII5007_over_OIII4959_Ratio_LinCon = -1000
                    Failed_OIII5007_over_OIII4959_Ratio.append(ObjectName)

            if Spectral_Lines_For_Analysis_Air['HeII4686'][16] in Exception_Values or Spectral_Lines_For_Analysis_Air['Halpha'][16] in Exception_Values:
                HeII_over_Halpha_Ratio_LinCon = -1000
            else:
                try:
                    HeII_over_Halpha_Ratio_LinCon = np.log10(
                        Spectral_Lines_For_Analysis_Air['HeII4686'][16] / Spectral_Lines_For_Analysis_Air['Halpha'][16])
                except FloatingPointError:
                    if debug:
                        print("\nlog10 error in the determination of HeII_over_Halpha_LinCon:\n"
                              "Likely cause is one or both lines measured in absorption."
                              "\nSetting value to -1000 and continuing.")
                    HeII_over_Halpha_Ratio_LinCon = -1000
                    Failed_HeII_over_Halpha_Ratio.append(ObjectName)
                except RuntimeWarning:
                    if debug:
                        print("\nlog10 error in the determination of lin_con_HeII_over_Halpha:\n"
                              "Looks like a zero got in there somewhere"
                              "\nSetting value to -1000 and continuing.")
                    HeII_over_Halpha_Ratio_LinCon = -1000
                    Failed_HeII_over_Halpha_Ratio.append(ObjectName)

            if HeII_over_Halpha_Ratio_LinCon == math.inf or NII_over_Halpha_Ratio_LinCon == -1 * math.inf:
                print(f"Looks like a cheeky zero still managed to squeak through, setting NII/Halpha to -1000 "
                      f"for {ObjectName}")
                HeII_over_Halpha_Ratio_LinCon = -1000
                Failed_NII_over_Halpha_Line_Ratio.append(ObjectName)

            if OIII_over_Hbeta_Ratio_LinCon == math.inf or OIII_over_Hbeta_Ratio_LinCon == -1 * math.inf:
                print(f"Looks like a cheeky zero still managed to squeak through, setting OIII/Hbeta to -1000 "
                      f"for {ObjectName}")
                OIII_over_Hbeta_Ratio_LinCon = -1000
                Failed_OIII_over_Hbeta_Line_Ratio.append(ObjectName)

            if SII_over_Halpha_Ratio_LinCon == math.inf or SII_over_Halpha_Ratio_LinCon == -1 * math.inf:
                print(f"Looks like a cheeky zero still managed to squeak through, setting SII/Halpha to -1000 "
                      f"for {ObjectName}")
                SII_over_Halpha_Ratio_LinCon = -1000
                Failed_SII_over_Halpha_Line_Ratio.append(ObjectName)

            if OI_over_Halpha_Ratio_LinCon == math.inf or OI_over_Halpha_Ratio_LinCon == -1 * math.inf:
                print(f"Looks like a cheeky zero still managed to squeak through, setting OI/Halpha to -1000 "
                      f"for {ObjectName}")
                OI_over_Halpha_Ratio_LinCon = -1000
                Failed_OI_over_Halpha_Line_Ratio.append(ObjectName)

            if OIII5007_over_OIII4959_Ratio_LinCon == math.inf or OIII5007_over_OIII4959_Ratio_LinCon == -1 * math.inf:
                print(f"Looks like a cheeky zero still managed to squeak through, setting OIII5007/OIII4959 to -1000 "
                      f"for {ObjectName}")
                OIII5007_over_OIII4959_Ratio_LinCon = -1000
                Failed_OIII5007_over_OIII4959_Ratio.append(ObjectName)

            if OIII5007_over_OIII4959_Ratio_LinCon == math.inf or OIII5007_over_OIII4959_Ratio_LinCon == -1 * math.inf:
                print(f"Looks like a cheeky zero still managed to squeak through, setting OIII5007/OIII4959 to -1000 "
                      f"for {ObjectName}")
                OIII5007_over_OIII4959_Ratio_LinCon = -1000
                Failed_OIII5007_over_OIII4959_Ratio.append(ObjectName)

            if debug:
                print("\n")
                print(ObjectName)
                print(OIII_over_Hbeta_Ratio_LinCon)
                print(NII_over_Halpha_Ratio_LinCon)
                print(SII_over_Halpha_Ratio_LinCon)
                print(OI_over_Halpha_Ratio_LinCon)
                print(OIII_over_Hbeta_Ratio_LinCon)
                print(Spectral_Lines_For_Analysis_Air['[NII]6584'][16])
                print(Spectral_Lines_For_Analysis_Air['Halpha'][16])
                print("\n")

            ##############################
            # Lick Indices Calculation
            ##############################

            try:
                Lick_Index_Data = Hirogen_Functions.lick_index_calculation(wave=wave.value, flux=flux.value, err=error)
            except:
                print("Lick Index Calculation Failure")
                Lick_Index_Data["d4000_n"] = -500
                Lick_Index_Data["d4000_n_err"] = -500
                Lick_Index_Data["lick_hd_a"] = -500
                Lick_Index_Data["lick_hg_a_err"] -500
                Failed_Lick.append(ObjectName)

            ##############################
            # NaN Screening
            ##############################

            for index in Lick_Index_Data:
                Lick_Index_Data[index] = -1000 if np.isnan(Lick_Index_Data[index]) else Lick_Index_Data[index]

            D4000 = Lick_Index_Data["d4000_n"]
            D4000_Err = Lick_Index_Data["d4000_n_err"]
            Lick_HDelta_Index = Lick_Index_Data["lick_hd_a"]
            Lick_HDelta_Index_Err = Lick_Index_Data["lick_hd_a_err"]
            Lick_HGamma_Index = Lick_Index_Data["lick_hg_a"]
            Lick_HGamma_Index_Err = Lick_Index_Data["lick_hg_a_err"]

            ##############################
            # Additional Object Flagging
            ##############################

            if Spectral_Lines_For_Analysis_Air['Halpha'][8] > -3: # Flags on the strength of weak Balmer emission
                Quiescent_Flag = 1
            else:
                Quiescent_Flag = 0

            if Lick_HDelta_Index == -1000:
                Balmer_Strong_Flag = -1
                Balmer_Moderately_Strong_Flag = -1
            else:

                if Lick_HDelta_Index - Lick_HDelta_Index_Err > 4:
                    Balmer_Strong_Flag = 1
                else:
                    Balmer_Strong_Flag = 0

                if Lick_HDelta_Index - Lick_HDelta_Index_Err > 1.31:
                    Balmer_Moderately_Strong_Flag = 1
                else:
                    Balmer_Moderately_Strong_Flag = 0

            NERSC_Storage_File.write(
                f"{ObjectName}\t"
                f"{Date}\t{Tile}\t{Spectrograph}\t{Fiber}\t{FiberStatus}\t"
                f"{Spectrum_Number}\t"
                f"{RA}\t"
                f"{DEC}\t"
                f"{Redshift}\t{Redshift_Err}\t{Extinction}\t"
                f"{Spectral_Lines_For_Analysis_Air['Halpha'][8]}\t{Spectral_Lines_For_Analysis_Air['Hbeta'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['Hgamma'][8]}\t{Spectral_Lines_For_Analysis_Air['Hdelta'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[NII]6548'][8]}\t{Spectral_Lines_For_Analysis_Air['[NII]6584'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeVII]3759'][8]}\t{Spectral_Lines_For_Analysis_Air['[FeVII]5160'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeVII]5722'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeVII]6088'][8]}\t{Spectral_Lines_For_Analysis_Air['[FeX]6376'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeXI]7894'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][8]}\t{Spectral_Lines_For_Analysis_Air['[OI]6300'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[OII]3728'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[OIII]4959'][8]}\t{Spectral_Lines_For_Analysis_Air['[OIII]5007'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['HeI4478'][8]}\t{Spectral_Lines_For_Analysis_Air['HeII4686'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['NaID'][8]}\t{Spectral_Lines_For_Analysis_Air['[SII]6716'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[SII]6731'][8]}\t{Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][8]}\t"
                f"{Spectral_Lines_For_Analysis_Air['Halpha'][16]}\t{Spectral_Lines_For_Analysis_Air['Hbeta'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['Hgamma'][16]}\t{Spectral_Lines_For_Analysis_Air['Hdelta'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[NII]6548'][16]}\t{Spectral_Lines_For_Analysis_Air['[NII]6584'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeVII]3759'][16]}\t{Spectral_Lines_For_Analysis_Air['[FeVII]5160'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeVII]5722'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeVII]6088'][16]}\t{Spectral_Lines_For_Analysis_Air['[FeX]6376'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeXI]7894'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[FeXIV]5304'][16]}\t{Spectral_Lines_For_Analysis_Air['[OI]6300'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[OII]3728'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[OIII]4959'][16]}\t{Spectral_Lines_For_Analysis_Air['[OIII]5007'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['HeI4478'][16]}\t{Spectral_Lines_For_Analysis_Air['HeII4686'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['NaID'][16]}\t{Spectral_Lines_For_Analysis_Air['[SII]6716'][16]}\t"
                f"{Spectral_Lines_For_Analysis_Air['[SII]6731'][16]}\t{Spectral_Lines_For_Analysis_Air['[SII]6716,6731'][16]}\t"
                f"{NII_over_Halpha_Ratio_LinCon}\t{OIII_over_Hbeta_Ratio_LinCon}\t{SII_over_Halpha_Ratio_LinCon}\t"
                f"{OI_over_Halpha_Ratio_LinCon}\t{OIII5007_over_OIII4959_Ratio_LinCon}\t{HeII_over_Halpha_Ratio_LinCon}\t"
                f"{D4000}\t{D4000_Err}\t{Lick_HGamma_Index}\t{Lick_HGamma_Index_Err}\t{Lick_HDelta_Index}\t{Lick_HDelta_Index_Err}\t"
                f"{Strong_FeVII_Flag}\t{Strong_FeX_Flag}\t{Strong_FeXI_Flag}\t{Strong_FeXIV_Flag}\t"
                f"{All_Flags_Percentage}\t{Counted_Flags_Percentage}\t"
                f"{Quiescent_Flag}\t{Balmer_Strong_Flag}\t{Balmer_Moderately_Strong_Flag}\t{Candidate_Score}\t"
                f"{G_FiberFlux}\t{G_Flux_Ivar}\t{R_FiberFlux}\t{R_Flux_Ivar}\t{Z_FiberFlux}\t{Z_Flux_Ivar}\t{W1_Flux}\t{W1_Flux_Ivar}\t{W2_Flux}\t{W2_Flux_Ivar}\t"
                f"{Identification_ID}\t{Settings_Config}\n"
            )

            # Variable iteration
            Spectrum_Number += 1

            #######
            # Candidate Filtering
            #######

            if Candidate_Score >= Candidate_Threshold or (Strong_FeVII_Flag+Strong_FeX_Flag+Strong_FeXI_Flag+Strong_FeXIV_Flag)> 0:
                Candidate = True
                Candidate_Count += 1
            else:
                Candidate = False

            ObjectCount += 1
            Successful_Objects_Count += 1


    NERSC_Storage_File.close() #Closing the storage file

    print(f"\n\nTotal number of DESI spectra processed: {Total_Number_of_Spectra}")
    print(f"Total number of DESI Target spectra considered: {Total_Number_of_Targets}")
    print(f"Total number of BGS Targets: {Total_Number_of_BGS_Targets}")
    print(f"Total number of spectra meeting examination cuts: {Total_Object_Count}")

    endT = time.time()
    executionT = endT - startT

    print("Done")
    print(f"Script execution time: {executionT:.2f}s")

else:
    print('This cell is for NERSC based DESI analysis only')

This cell is for NERSC based DESI analysis only
